In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

BASE_DIR = "/content/drive/MyDrive/AML Challenge"
os.chdir(BASE_DIR)
print("Current working directory:", os.getcwd())


Current working directory: /content/drive/MyDrive/AML Challenge


# Pre-Processing

In [3]:
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np, pandas as pd
# ------------------------------------------------------
# 1. Device and data loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

train_data = np.load("train.npz")
test_data  = np.load("test.clean.npz")

tx_train = train_data["captions/embeddings"]   # (125000, 1024)
im_train = train_data["images/embeddings"]     # (25000, 1536)
tx_test  = test_data["captions/embeddings"]    # (1500, 1024)

print("Train shapes:", tx_train.shape, im_train.shape)
print("Test shape:", tx_test.shape)

# ------------------------------------------------------
# 2. Match each caption to its corresponding image
# ------------------------------------------------------
repeat_factor = len(tx_train) // len(im_train)   # 5 captions per image
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)

Using device: cuda
Train shapes: (125000, 1024) (25000, 1536)
Test shape: (1500, 1024)


# Experiment 1: got score as 0.81780
# Residual-Orthogonal + Contrastive version

In [ ]:
# ------------------------------------------------------
# 2. Match each caption to its corresponding image
# ------------------------------------------------------
repeat_factor = len(tx_train) // len(im_train)   # 5 captions per image
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)

# ------------------------------------------------------
# 3. Convert to tensors + center + normalize
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

# Center + normalize (same mean for test)
tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t.mean(0, keepdim=True)

tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t = F.normalize(im_train_t - im_mean, p=2, dim=1)
tx_test_t  = F.normalize(tx_test_t  - tx_mean, p=2, dim=1)

# Expand image embeddings for each caption
im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)

# ------------------------------------------------------
# 4. Compute Orthogonal Procrustes base (R)
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh  # (1536 × 1024)

print("Computed orthogonal base R:", R.shape)

# ------------------------------------------------------
# 5. Define Residual-Orthogonal Translator
# ------------------------------------------------------
class ResidualTranslator(nn.Module):
    def __init__(self, R_init, input_dim=1024, hidden_dim=1024, output_dim=1536):
        super().__init__()
        self.register_buffer("R", R_init)
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        base = x @ self.R.T
        res = self.residual(x)
        out = F.normalize(base + res, p=2, dim=1)
        return out

# ------------------------------------------------------
# 6. Define losses
# ------------------------------------------------------
def contrastive_loss(pred, target, tau=0.07):
    sims = pred @ target.T / tau
    labels = torch.arange(pred.size(0), device=device)
    return F.cross_entropy(sims, labels)

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=0.2
)

# ------------------------------------------------------
# 7. DataLoader
# ------------------------------------------------------
train_dataset = TensorDataset(tx_train_t, im_train_exp)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)

# ------------------------------------------------------
# 8. Initialize model + optimizer
# ------------------------------------------------------
model = ResidualTranslator(R.detach().clone()).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

# ------------------------------------------------------
# 9. Training loop
# ------------------------------------------------------
EPOCHS = 50
print("\nTraining Residual-Orthogonal Translator...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    for x_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()

        y_pred = model(x_batch)
        idx = torch.randperm(x_batch.size(0), device=device)
        y_neg = y_batch[idx]

        loss_cos = contrastive_loss(y_pred, y_batch)
        loss_tri = triplet_loss_fn(y_pred, y_batch, y_neg)
        loss = 0.6 * loss_cos + 0.4 * loss_tri

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch:02d}: avg loss = {avg_loss:.4f}")

torch.save(model.state_dict(), "residual_orthogonal.pth")
print("\n✅ Training completed and model saved as residual_orthogonal.pth")

# ------------------------------------------------------
# 10. Inference for submission
# ------------------------------------------------------
model.eval()
with torch.no_grad():
    tx_test_n = F.normalize(tx_test_t, p=2, dim=1)
    preds = model(tx_test_n).cpu().numpy()

# ------------------------------------------------------
# 11. Save submission file
# ------------------------------------------------------
test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_residual.csv", index=False)

print("✅ Saved submission_residual.csv")
print(submission.head(3))

Computed orthogonal base R: torch.Size([1536, 1024])

Training Residual-Orthogonal Translator...



Epoch 01: avg loss = 1.5180


Epoch 02: avg loss = 1.3856


Epoch 03: avg loss = 1.3359


Epoch 04: avg loss = 1.3061


...

Epoch 46: avg loss = 1.1360


Epoch 47: avg loss = 1.1351


Epoch 48: avg loss = 1.1343


Epoch 49: avg loss = 1.1329


Epoch 50: avg loss = 1.1321

✅ Training completed and model saved as residual_orthogonal.pth
✅ Saved submission_residual.csv
   id                                          embedding
0   1  [-0.010053236037492752, 0.014468204230070114, ...
1   2  [-0.03426738828420639, -0.030702663585543633, ...
2   3  [-0.0029787796083837748, -0.02091299369931221,...


# hard_negative approach

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
from google.colab import drive


# ------------------------------------------------------
# 3. Convert to tensors + center + normalize
#    (Using the exact preprocessing from your 0.81780 experiment)
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device) # For R
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

# Center + normalize (same mean for test)
tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True) # Mean of unique images

tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t  = F.normalize(tx_test_t  - tx_mean, p=2, dim=1)

# Expand image embeddings for each caption (for training)
im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)

print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 4. Compute Orthogonal Procrustes base (R)
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t_unique.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh  # (1536 × 1024)

print(f"Computed orthogonal base R: {R.shape}")

# ------------------------------------------------------
# 5. Define Residual-Orthogonal Translator
#    (Same model as your 0.81780 experiment)
# ------------------------------------------------------
class ResidualTranslator(nn.Module):
    def __init__(self, R_init, input_dim=1024, hidden_dim=1024, output_dim=1536):
        super().__init__()
        self.register_buffer("R", R_init)
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        base = x @ self.R.T
        res = self.residual(x)
        out = F.normalize(base + res, p=2, dim=1)
        return out

# ------------------------------------------------------
# 6. Define losses
# ------------------------------------------------------
def contrastive_loss(pred, target, tau=0.07):
    # pred: (B, D), target: (B, D)
    sims = pred @ target.T / tau
    labels = torch.arange(pred.size(0), device=device)
    return F.cross_entropy(sims, labels)

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=0.2
)

# ------------------------------------------------------
# 7. DataLoader
# ------------------------------------------------------
train_dataset = TensorDataset(tx_train_t, im_train_exp)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)

# ------------------------------------------------------
# 8. Initialize model + optimizer
# ------------------------------------------------------
model = ResidualTranslator(R.detach().clone()).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scaler = torch.cuda.amp.GradScaler() # For mixed precision

# ------------------------------------------------------
# 9. Training loop with HARD-NEGATIVE MINING
# ------------------------------------------------------
EPOCHS = 30
TAU = 0.07 # Temperature for contrastive loss
LOSS_WEIGHT_TRIPLET = 0.3 # Weight for the triplet loss term
LOSS_WEIGHT_CONTRASTIVE = 0.7 # Weight for the contrastive loss term

print("\nTraining Residual-Orthogonal Translator with Hard-Negative Mining...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    for x_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            # --- Forward pass ---
            y_pred = model(x_batch) # (B, 1536)

            # --- 1. Contrastive Loss (InfoNCE) ---
            # This is the main loss, computes (B, B) similarity matrix
            sims_with_tau = y_pred @ y_batch.T / TAU
            labels = torch.arange(y_pred.size(0), device=device)
            loss_cos = F.cross_entropy(sims_with_tau, labels)

            # --- 2. Hard-Negative Mining for Triplet Loss ---
            with torch.no_grad():
                # We need similarities *without* temperature scaling
                sims_no_tau = y_pred @ y_batch.T # (B, B)

                # Mask out the positive sample (the diagonal)
                sims_no_tau.masked_fill_(
                    torch.eye(y_batch.size(0), device=device, dtype=torch.bool),
                    -float('inf')
                )

                # Find the highest-similarity (hardest) negative for each sample
                hard_neg_idx = sims_no_tau.argmax(dim=1) # (B)

            y_hard_neg = y_batch[hard_neg_idx] # (B, 1536)

            # --- 3. Triplet Loss (with hard negatives) ---
            loss_tri = triplet_loss_fn(y_pred, y_batch, y_hard_neg)

            # --- 4. Combine losses ---
            loss = (LOSS_WEIGHT_CONTRASTIVE * loss_cos) + (LOSS_WEIGHT_TRIPLET * loss_tri)

        # --- Backward pass ---
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch:02d}: avg loss = {avg_loss:.4f}")

torch.save(model.state_dict(), "residual_hard_negative.pth")
print("\n✅ Training completed and model saved as residual_hard_negative.pth")

# ------------------------------------------------------
# 10. Inference for submission
# ------------------------------------------------------
model.eval()
with torch.no_grad():
    # We normalized tx_test_t during preprocessing
    preds = model(tx_test_t).cpu().numpy()

# ------------------------------------------------------
# 11. Save submission file
# ------------------------------------------------------
test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_hard_negative.csv", index=False)

print("✅ Saved submission_hard_negative.csv")
print(submission.head(3))


Data preprocessed and normalized.


/tmp/ipython-input-908034218.py:89: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # For mixed precision


Computed orthogonal base R: torch.Size([1536, 1024])

Training Residual-Orthogonal Translator with Hard-Negative Mining...



Epoch 1/30:   0%|          | 0/245 [00:00<?, ?it/s]/tmp/ipython-input-908034218.py:109: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 01: avg loss = 1.8259


...

Epoch 26: avg loss = 1.4079


Epoch 27: avg loss = 1.4045


Epoch 28: avg loss = 1.4034


Epoch 29: avg loss = 1.4006


Epoch 30: avg loss = 1.3981

✅ Training completed and model saved as residual_hard_negative.pth
✅ Saved submission_hard_negative.csv
   id                                          embedding
0   1  [-0.010386133566498756, 0.015329176560044289, ...
1   2  [-0.03257265314459801, -0.027175934985280037, ...
2   3  [-0.00526655837893486, -0.014171533286571503, ...


# Experiment 3: Residual + Hard Negative

In [ ]:
# ======================================================
#  AML CHALLENGE — Residual-Orthogonal Translator v3
#  (Hard Negatives + Orthogonal Reg + CSLS + Ensemble)
# ======================================================

import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np, pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ------------------------------------------------------
# 1. Data preprocessing  (same as before)
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)
tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)
print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 2. Orthogonal Procrustes base
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t_unique.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh
print(f"Computed orthogonal base R: {R.shape}")

# ------------------------------------------------------
# 3. Model
# ------------------------------------------------------
class ResidualTranslator(nn.Module):
    def __init__(self, R_init, input_dim=1024, hidden_dim=1024, output_dim=1536):
        super().__init__()
        self.register_buffer("R", R_init)
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        base = x @ self.R.T
        res  = self.residual(x)
        return F.normalize(base + res, p=2, dim=1)

# Orthogonality penalty
def orthogonal_penalty(W):
    WT_W = W.T @ W
    I = torch.eye(WT_W.shape[0], device=W.device)
    return ((WT_W - I)**2).mean()

# ------------------------------------------------------
# 4. Losses
# ------------------------------------------------------
triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=0.2
)

# ------------------------------------------------------
# 5. Dataloader
# ------------------------------------------------------
train_dataset = TensorDataset(tx_train_t, im_train_exp)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)

# ------------------------------------------------------
# 6. Train
# ------------------------------------------------------
def train_model(seed=42, save_path="residual_v3.pth"):
    torch.manual_seed(seed)
    model = ResidualTranslator(R.detach().clone()).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scaler = torch.amp.GradScaler("cuda")

    EPOCHS = 30
    TAU = 0.07
    LOSS_W_CONTR = 0.7
    LOSS_W_TRIP = 0.3
    ORTHO_W = 1e-4

    print(f"\nTraining Residual-Orthogonal Translator (seed={seed})...\n")
    for epoch in range(1, EPOCHS + 1):
        model.train()
        total_loss = 0.0
        for x_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
            optimizer.zero_grad()
            with torch.amp.autocast("cuda"):
                y_pred = model(x_batch)
                sims = y_pred @ y_batch.T / TAU
                labels = torch.arange(y_pred.size(0), device=device)
                loss_con = F.cross_entropy(sims, labels)

                with torch.no_grad():
                    sims_no_tau = y_pred @ y_batch.T
                    sims_no_tau.masked_fill_(torch.eye(y_batch.size(0), device=device, dtype=torch.bool), -float('inf'))
                    hard_idx = sims_no_tau.argmax(dim=1)
                y_hard = y_batch[hard_idx]
                loss_tri = triplet_loss_fn(y_pred, y_batch, y_hard)

                loss_ortho = orthogonal_penalty(model.R)
                loss = (LOSS_W_CONTR * loss_con) + (LOSS_W_TRIP * loss_tri) + (ORTHO_W * loss_ortho)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
        print(f"Epoch {epoch:02d}: avg loss = {total_loss/len(train_loader):.4f}")

    torch.save(model.state_dict(), save_path)
    print(f"\n✅ Model saved as {save_path}\n")
    return model

# ------------------------------------------------------
# 7. Train + Ensemble (3 seeds)
# ------------------------------------------------------
models = []
for seed in [42, 77, 123]:
    model = train_model(seed, save_path=f"residual_v3_seed{seed}.pth")
    models.append(model)

# ------------------------------------------------------
# 8. Inference (ensemble-averaged)
# ------------------------------------------------------
@torch.no_grad()
def predict_ensemble(models, tx_test_t):
    preds_all = []
    for model in models:
        model.eval()
        preds_all.append(F.normalize(model(tx_test_t), p=2, dim=1).cpu().numpy())
    return np.mean(preds_all, axis=0)

preds = predict_ensemble(models, tx_test_t)

# ------------------------------------------------------
# 9. CSLS re-ranking (optional at test time)
# ------------------------------------------------------
@torch.no_grad()
def apply_csls(preds, im_base, k=10):
    preds = F.normalize(torch.tensor(preds, device=device), p=2, dim=1)
    im_base = F.normalize(im_base, p=2, dim=1)
    sim = preds @ im_base.T
    knn_q = torch.topk(sim, k=k, dim=1).values.mean(1, keepdim=True)
    knn_d = torch.topk(sim, k=k, dim=0).values.mean(0, keepdim=True)
    return (2 * sim - knn_q - knn_d).cpu().numpy()

csls_sim = apply_csls(preds, im_train_t_unique)

# ------------------------------------------------------
# 10. Save submission
# ------------------------------------------------------
test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_residual_v3.csv", index=False)
print("✅ submission_residual_v3.csv saved.")


Using device: cuda
Data preprocessed and normalized.
Computed orthogonal base R: torch.Size([1536, 1024])

Training Residual-Orthogonal Translator (seed=42)...



Epoch 01: avg loss = 1.8249


Epoch 02: avg loss = 1.6702


Epoch 03: avg loss = 1.6105


Epoch 04: avg loss = 1.5747


...

Epoch 26: avg loss = 1.4077


Epoch 27: avg loss = 1.4061


Epoch 28: avg loss = 1.4028


Epoch 29: avg loss = 1.4008


Epoch 30: avg loss = 1.3987

✅ Model saved as residual_v3_seed123.pth

✅ submission_residual_v3.csv saved.


##

#  Model: Residual-Orthogonal TRANSFORMER Translator



In [ ]:

# ------------------------------------------------------
# 1. Device and data loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_data = np.load("train.npz")
test_data  = np.load("test.clean.npz")

tx_train = train_data["captions/embeddings"]
im_train = train_data["images/embeddings"]
tx_test  = test_data["captions/embeddings"]

repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)
print(f"Train shapes: {tx_train.shape}, {im_train.shape}, {im_train_expanded.shape}")
print(f"Test shape: {tx_test.shape}")

# ------------------------------------------------------
# 2. Data preprocessing (Centering + Normalization)
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)
tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)
print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 3. Orthogonal Procrustes base (R)
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t_unique.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh
print(f"Computed orthogonal base R: {R.shape}")

# ------------------------------------------------------
# 4. Model: Residual-Orthogonal (Simple MLP)
#    (Your proven 0.81780-scoring model)
# ------------------------------------------------------
class ResidualTranslator(nn.Module):
    def __init__(self, R_init, input_dim=1024, hidden_dim=1024, output_dim=1536):
        super().__init__()
        self.register_buffer("R", R_init)
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        base = x @ self.R.T
        res = self.residual(x)
        return F.normalize(base + res, p=2, dim=1)

# ------------------------------------------------------
# 5. Loss Functions (Contrastive + CORRECT Triplet)
# ------------------------------------------------------
TAU = 0.07 # Temperature for contrastive loss
LOSS_WEIGHT_CONTRASTIVE = 0.7
LOSS_WEIGHT_TRIPLET = 0.3

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=0.2
)

# ------------------------------------------------------
# 6. Validation split (10%)
#    (We MUST pass the original image index)
# ------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)
idx_cpu = torch.randperm(N, device="cpu") # Use CPU for randperm
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

# Get the corresponding image index (0-24999) for each caption
img_indices_train = (train_idx // 5).to(device)
img_indices_val = (val_idx // 5).to(device)

# --- Create datasets ---
tx_val_t, im_val_t = tx_train_t[val_idx], im_train_exp[val_idx]
tx_train_t_sub, im_train_exp_sub = tx_train_t[train_idx], im_train_exp[train_idx]

# --- Dataloaders now include the image index ---
train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# ------------------------------------------------------
# 7. Recall@K utility (for validation)
# ------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, repeat_factor=5, ks=(1, 5, 10, 50), k_csls=10):
    """
    Evaluates Text-to-Image Recall@K using CSLS re-ranking.
    - tx_queries: (N_queries, 1024) tensor of text embeddings
    - im_database: (N_images, 1536) tensor of ALL unique image embeddings
    - query_img_indices: (N_queries,) tensor of GT image indices (0-24999)
    """
    model.eval()

    # --- 1. Predict caption embeddings ---
    preds_list = []
    for i in range(0, len(tx_queries), 1024):
        chunk = tx_queries[i:i+1024]
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0) # (N_queries, 1536)

    # --- 2. Compute Cosine Similarity Matrix ---
    sim = preds @ im_database.T # (N_queries, N_images)

    # --- 3. Compute CSLS re-ranking scores ---
    knn_q = torch.topk(sim, k=k_csls, dim=1).values
    mean_knn_q = knn_q.mean(1, keepdim=True) # (N_queries, 1)

    knn_d = torch.topk(sim.T, k=k_csls, dim=1).values
    mean_knn_d = knn_d.mean(1, keepdim=True).T # (1, N_images)

    csls_sim = 2 * sim - mean_knn_q - mean_knn_d # (N_queries, N_images)

    # --- 4. Get Ground Truth (already passed in) ---
    gt = query_img_indices # (N_queries,)

    # --- 5. Calculate R@K ---
    top_indices = torch.argsort(csls_sim, dim=1, descending=True)

    recalls = {}
    for k in ks:
        top_k_preds = top_indices[:, :k]
        correct_in_top_k = (top_k_preds == gt.unsqueeze(1)).any(dim=1)
        recall_at_k = correct_in_top_k.float().mean().item()
        recalls[f"R@{k}"] = recall_at_k

    return recalls

# ------------------------------------------------------
# 8. Training Loop
# ------------------------------------------------------
EPOCHS = 40
LR = 1e-4
SAVE_PATH = "correct_hard_negative_best.pth"

# Validation set for monitoring
val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique # Use ALL unique images as the database

model = ResidualTranslator(R.detach().clone()).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler() # Mixed precision

best_r10 = 0.0
print(f"\nTraining with CORRECT Hard-Negative Mining...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            # --- Forward pass ---
            y_pred = model(x_batch) # (B, 1536)

            # --- 1. Contrastive Loss (aligns pairs) ---
            sims_with_tau = y_pred @ y_batch.T / TAU
            labels = torch.arange(y_pred.size(0), device=device)
            loss_con = F.cross_entropy(sims_with_tau, labels)

            # --- 2. CORRECT Hard-Negative Mining ---
            with torch.no_grad():
                # (B, B) matrix of pairwise similarities
                sims_no_tau = y_pred @ y_batch.T

                # (B, B) mask where True means it's a POSITIVE pair
                positive_mask = (img_indices.unsqueeze(1) == img_indices.unsqueeze(0))

                # Mask out ALL positives (not just diagonal)
                sims_no_tau.masked_fill_(positive_mask, -float('inf'))

                # The argmax is now guaranteed to be a TRUE negative
                hard_neg_idx = sims_no_tau.argmax(dim=1)

            y_hard_neg = y_batch[hard_neg_idx]

            # --- 3. Triplet Loss (now correct) ---
            loss_tri = triplet_loss_fn(y_pred, y_batch, y_hard_neg)

            # --- 4. Combine losses ---
            loss = (LOSS_WEIGHT_CONTRASTIVE * loss_con) + (LOSS_WEIGHT_TRIPLET * loss_tri)

        # --- Backward pass ---
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    scheduler.step()
    avg_loss = total_loss / len(train_loader)

    # ----- Validation -----
    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset)
    print(f"Epoch {epoch:02d}: loss={avg_loss:.4f} | "
          f"R@1={rec['R@1']:.4f}  R@5={rec['R@5']:.4f}  "
          f"R@10={rec['R@10']:.4f}  R@50={rec['R@50']:.4f}")

    if rec['R@10'] > best_r10:
        best_r10 = rec['R@10']
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"✅ Best model saved (epoch {epoch}, R@10={best_r10:.4f})")

print(f"\n🎯 Best R@10 on validation = {best_r10:.4f}\nModel saved as {SAVE_PATH}")

# ------------------------------------------------------
# 9. Inference + Submission
# ------------------------------------------------------
print(f"Loading best model from {SAVE_PATH} for inference...")
model = ResidualTranslator(R.detach().clone()).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    # tx_test_t was already preprocessed
    preds_list = []
    for i in range(0, len(tx_test_t), 1024):
        chunk = tx_test_t[i:i+1024]
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_correct_hard_negative.csv", index=False)
print("\n✅ submission_correct_hard_negative.csv saved successfully.")


Using device: cuda
Train shapes: (125000, 1024), (25000, 1536), (125000, 1536)
Test shape: (1500, 1024)
Data preprocessed and normalized.


/tmp/ipython-input-2102333727.py:161: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # Mixed precision


Computed orthogonal base R: torch.Size([1536, 1024])
Train pairs: 112500, Validation pairs: 12500

Training with CORRECT Hard-Negative Mining...



Epoch 1/40:   0%|          | 0/220 [00:00<?, ?it/s]/tmp/ipython-input-2102333727.py:173: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 01: loss=1.8393 | R@1=0.1730  R@5=0.3746  R@10=0.4700  R@50=0.7056
✅ Best model saved (epoch 1, R@10=0.4700)


...

Epoch 36: loss=1.4100 | R@1=0.1984  R@5=0.4194  R@10=0.5226  R@50=0.7590


Epoch 37: loss=1.4085 | R@1=0.1994  R@5=0.4194  R@10=0.5222  R@50=0.7582


Epoch 38: loss=1.4093 | R@1=0.1986  R@5=0.4186  R@10=0.5224  R@50=0.7584


Epoch 39: loss=1.4099 | R@1=0.1986  R@5=0.4188  R@10=0.5224  R@50=0.7586


Epoch 40: loss=1.4086 | R@1=0.1986  R@5=0.4188  R@10=0.5224  R@50=0.7584

🎯 Best R@10 on validation = 0.5242
Model saved as correct_hard_negative_best.pth
Loading best model from correct_hard_negative_best.pth for inference...
Best model loaded.

✅ submission_correct_hard_negative.csv saved successfully.


# Tuning to previous code and enhancement

In [ ]:
# ------------------------------------------------------
# 1. Device and data loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_data = np.load("train.npz")
test_data  = np.load("test.clean.npz")

tx_train = train_data["captions/embeddings"]
im_train = train_data["images/embeddings"]
tx_test  = test_data["captions/embeddings"]

repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)
print(f"Train shapes: {tx_train.shape}, {im_train.shape}, {im_train_expanded.shape}")
print(f"Test shape: {tx_test.shape}")

# ------------------------------------------------------
# 2. Data preprocessing (Centering + Normalization)
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)
tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)
print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 3. Orthogonal Procrustes base (R)
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t_unique.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh
print(f"Computed orthogonal base R: {R.shape}")

# ------------------------------------------------------
# 4. NEW MODEL: EnhancedResidualTranslator
#    (Deeper, more powerful residual block)
# ------------------------------------------------------
class EnhancedResidualTranslator(nn.Module):
    def __init__(self, R_init, input_dim=1024, hidden_dim=2048, output_dim=1536, dropout=0.1):
        super().__init__()
        self.register_buffer("R", R_init)
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        base = x @ self.R.T
        res = self.residual(x)
        return F.normalize(base + res, p=2, dim=1)

# ------------------------------------------------------
# 5. Loss Functions (Unchanged)
# ------------------------------------------------------
TAU = 0.07 # Temperature for contrastive loss
LOSS_WEIGHT_CONTRASTIVE = 0.7
LOSS_WEIGHT_TRIPLET = 0.3

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=0.2
)

# ------------------------------------------------------
# 6. Validation split (Unchanged)
# ------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)
idx_cpu = torch.randperm(N, device="cpu") # Use CPU for randperm
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_train = (train_idx // 5).to(device)
img_indices_val = (val_idx // 5).to(device)

tx_val_t, im_val_t = tx_train_t[val_idx], im_train_exp[val_idx]
tx_train_t_sub, im_train_exp_sub = tx_train_t[train_idx], im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# ------------------------------------------------------
# 7. Recall@K utility (Unchanged)
# ------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, repeat_factor=5, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    preds_list = []
    for i in range(0, len(tx_queries), 1024):
        chunk = tx_queries[i:i+1024]
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0) # (N_queries, 1536)

    sim = preds @ im_database.T # (N_queries, N_images)

    knn_q = torch.topk(sim, k=k_csls, dim=1).values
    mean_knn_q = knn_q.mean(1, keepdim=True) # (N_queries, 1)

    knn_d = torch.topk(sim.T, k=k_csls, dim=1).values
    mean_knn_d = knn_d.mean(1, keepdim=True).T # (1, N_images)

    csls_sim = 2 * sim - mean_knn_q - mean_knn_d # (N_queries, N_images)

    gt = query_img_indices # (N_queries,)

    top_indices = torch.argsort(csls_sim, dim=1, descending=True)

    recalls = {}
    for k in ks:
        top_k_preds = top_indices[:, :k]
        correct_in_top_k = (top_k_preds == gt.unsqueeze(1)).any(dim=1)
        recall_at_k = correct_in_top_k.float().mean().item()
        recalls[f"R@{k}"] = recall_at_k

    return recalls

# ------------------------------------------------------
# 8. Training Loop (Unchanged, but with new model)
# ------------------------------------------------------
EPOCHS = 40
LR = 1e-4
SAVE_PATH = "enhanced_hard_negative_best.pth" # New save path

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

# --- Initialize NEW model ---
model = EnhancedResidualTranslator(R.detach().clone(), hidden_dim=2048).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler() # Mixed precision

best_r10 = 0.0
print(f"\nTraining with Enhanced Model + Correct Hard-Negative Mining...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            y_pred = model(x_batch)

            sims_with_tau = y_pred @ y_batch.T / TAU
            labels = torch.arange(y_pred.size(0), device=device)
            loss_con = F.cross_entropy(sims_with_tau, labels)

            with torch.no_grad():
                sims_no_tau = y_pred @ y_batch.T
                positive_mask = (img_indices.unsqueeze(1) == img_indices.unsqueeze(0))
                sims_no_tau.masked_fill_(positive_mask, -float('inf'))
                hard_neg_idx = sims_no_tau.argmax(dim=1)

            y_hard_neg = y_batch[hard_neg_idx]
            loss_tri = triplet_loss_fn(y_pred, y_batch, y_hard_neg)

            loss = (LOSS_WEIGHT_CONTRASTIVE * loss_con) + (LOSS_WEIGHT_TRIPLET * loss_tri)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    scheduler.step()
    avg_loss = total_loss / len(train_loader)

    # ----- Validation -----
    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset)
    print(f"Epoch {epoch:02d}: loss={avg_loss:.4f} | "
          f"R@1={rec['R@1']:.4f}  R@5={rec['R@5']:.4f}  "
          f"R@10={rec['R@10']:.4f}  R@50={rec['R@50']:.4f}")

    if rec['R@10'] > best_r10:
        best_r10 = rec['R@10']
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"✅ Best model saved (epoch {epoch}, R@10={best_r10:.4f})")

print(f"\n🎯 Best R@10 on validation = {best_r10:.4f}\nModel saved as {SAVE_PATH}")

# ------------------------------------------------------
# 9. Inference + Submission
# ------------------------------------------------------
print(f"Loading best model from {SAVE_PATH} for inference...")
model = EnhancedResidualTranslator(R.detach().clone(), hidden_dim=2048).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in range(0, len(tx_test_t), 1024):
        chunk = tx_test_t[i:i+1024]
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_enhanced_hard_negative.csv", index=False)
print("\n✅ submission_enhanced_hard_negative.csv saved successfully.")


Using device: cuda
Train shapes: (125000, 1024), (25000, 1536), (125000, 1536)
Test shape: (1500, 1024)
Data preprocessed and normalized.


/tmp/ipython-input-2585142698.py:151: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # Mixed precision


Computed orthogonal base R: torch.Size([1536, 1024])
Train pairs: 112500, Validation pairs: 12500

Training with Enhanced Model + Correct Hard-Negative Mining...



Epoch 1/40:   0%|          | 0/220 [00:00<?, ?it/s]/tmp/ipython-input-2585142698.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 01: loss=2.1814 | R@1=0.1082  R@5=0.2682  R@10=0.3684  R@50=0.6446
✅ Best model saved (epoch 1, R@10=0.3684)


...

Epoch 36: loss=0.5315 | R@1=0.2444  R@5=0.4560  R@10=0.5586  R@50=0.7650


Epoch 37: loss=0.5302 | R@1=0.2436  R@5=0.4558  R@10=0.5578  R@50=0.7648


Epoch 38: loss=0.5287 | R@1=0.2440  R@5=0.4564  R@10=0.5584  R@50=0.7642


Epoch 39: loss=0.5279 | R@1=0.2440  R@5=0.4560  R@10=0.5572  R@50=0.7646


Epoch 40: loss=0.5281 | R@1=0.2444  R@5=0.4558  R@10=0.5572  R@50=0.7646

🎯 Best R@10 on validation = 0.5616
Model saved as enhanced_hard_negative_best.pth
Loading best model from enhanced_hard_negative_best.pth for inference...
Best model loaded.

✅ submission_enhanced_hard_negative.csv saved successfully.


# Patching Transformer

In [ ]:
# ------------------------------------------------------
# 1. Device and data loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_data = np.load("train.npz")
test_data  = np.load("test.clean.npz")

tx_train = train_data["captions/embeddings"]
im_train = train_data["images/embeddings"]
tx_test  = test_data["captions/embeddings"]

repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)
print(f"Train shapes: {tx_train.shape}, {im_train.shape}, {im_train_expanded.shape}")
print(f"Test shape: {tx_test.shape}")

# ------------------------------------------------------
# 2. Data preprocessing (Centering + Normalization)
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)
tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)
print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 3. Orthogonal Procrustes base (R)
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t_unique.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh
print(f"Computed orthogonal base R: {R.shape}")

# ------------------------------------------------------
# 4. NEW MODEL: ResidualPatchingTransformer
# ------------------------------------------------------
class ResidualPatchingTransformer(nn.Module):
    def __init__(self, R_init,
                 input_dim=1024,
                 output_dim=1536,
                 patch_size=64,
                 embed_dim=256,
                 depth=3,
                 nhead=8,
                 dropout=0.1):
        super().__init__()
        self.register_buffer("R", R_init)

        assert input_dim % patch_size == 0, "Input dim must be divisible by patch size"
        num_patches = input_dim // patch_size # 1024 // 64 = 16 patches

        # --- 1. Patching and Embedding ---
        # We use a single Linear layer to do both patching and embedding
        self.patch_embed = nn.Linear(input_dim, embed_dim * num_patches)
        self.pos_embed = nn.Parameter(torch.randn(1, num_patches, embed_dim) * 0.02)

        self.num_patches = num_patches
        self.embed_dim = embed_dim
        self.dropout = nn.Dropout(dropout)

        # --- 2. Transformer Encoder ---
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=nhead,
            dim_feedforward=embed_dim * 4,
            dropout=dropout,
            activation='gelu',
            batch_first=True,
            norm_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=depth)

        # --- 3. Output Projection ---
        self.norm = nn.LayerNorm(embed_dim)
        self.output_proj = nn.Linear(embed_dim * num_patches, output_dim)

    def forward(self, x):
        # 1. Base (Orthogonal) Path
        base = x @ self.R.T # (B, 1536)

        # 2. Residual (Transformer) Path

        # Patch and embed
        x_res = self.patch_embed(x) # (B, 1024) -> (B, 16*256)
        x_res = x_res.view(-1, self.num_patches, self.embed_dim) # (B, 16, 256)

        # Add positional embedding
        x_res = x_res + self.pos_embed # (B, 16, 256)
        x_res = self.dropout(x_res)

        # Pass through Transformer
        x_res = self.transformer(x_res) # (B, 16, 256)
        x_res = self.norm(x_res)

        # Flatten and project to output
        x_res = x_res.flatten(start_dim=1) # (B, 16*256)
        res = self.output_proj(x_res) # (B, 1536)

        # 3. Combine and Normalize
        return F.normalize(base + res, p=2, dim=1)

# ------------------------------------------------------
# 5. Loss Functions (Unchanged)
# ------------------------------------------------------
TAU = 0.07 # Temperature for contrastive loss
LOSS_WEIGHT_CONTRASTIVE = 0.7
LOSS_WEIGHT_TRIPLET = 0.3

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=0.2
)

# ------------------------------------------------------
# 6. Validation split (Unchanged)
# ------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)
idx_cpu = torch.randperm(N, device="cpu") # Use CPU for randperm
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_train = (train_idx // 5).to(device)
img_indices_val = (val_idx // 5).to(device)

tx_val_t, im_val_t = tx_train_t[val_idx], im_train_exp[val_idx]
tx_train_t_sub, im_train_exp_sub = tx_train_t[train_idx], im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# ------------------------------------------------------
# 7. Recall@K utility (Unchanged)
# ------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, repeat_factor=5, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    preds_list = []
    for i in range(0, len(tx_queries), 1024):
        chunk = tx_queries[i:i+1024]
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0) # (N_queries, 1536)

    sim = preds @ im_database.T # (N_queries, N_images)

    knn_q = torch.topk(sim, k=k_csls, dim=1).values
    mean_knn_q = knn_q.mean(1, keepdim=True) # (N_queries, 1)

    knn_d = torch.topk(sim.T, k=k_csls, dim=1).values
    mean_knn_d = knn_d.mean(1, keepdim=True).T # (1, N_images)

    csls_sim = 2 * sim - mean_knn_q - mean_knn_d # (N_queries, N_images)

    gt = query_img_indices # (N_queries,)

    top_indices = torch.argsort(csls_sim, dim=1, descending=True)

    recalls = {}
    for k in ks:
        top_k_preds = top_indices[:, :k]
        correct_in_top_k = (top_k_preds == gt.unsqueeze(1)).any(dim=1)
        recall_at_k = correct_in_top_k.float().mean().item()
        recalls[f"R@{k}"] = recall_at_k

    return recalls

# ------------------------------------------------------
# 8. Training Loop (Unchanged, but with new model)
# ------------------------------------------------------
EPOCHS = 40
LR = 1e-4
SAVE_PATH = "patching_transformer_best.pth" # New save path

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

# --- Initialize NEW model ---
model = ResidualPatchingTransformer(
    R_init=R.detach().clone(),
    patch_size=64,    # 1024 -> 16 patches
    embed_dim=256,    # Each patch becomes 256-dim
    depth=3,          # 3 Transformer layers
    nhead=8           # 8 attention heads
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler() # Mixed precision

best_r10 = 0.0
print(f"\nTraining with Patching Transformer + Correct Hard-Negative Mining...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            y_pred = model(x_batch)

            sims_with_tau = y_pred @ y_batch.T / TAU
            labels = torch.arange(y_pred.size(0), device=device)
            loss_con = F.cross_entropy(sims_with_tau, labels)

            with torch.no_grad():
                sims_no_tau = y_pred @ y_batch.T
                positive_mask = (img_indices.unsqueeze(1) == img_indices.unsqueeze(0))
                sims_no_tau.masked_fill_(positive_mask, -float('inf'))
                hard_neg_idx = sims_no_tau.argmax(dim=1)

            y_hard_neg = y_batch[hard_neg_idx]
            loss_tri = triplet_loss_fn(y_pred, y_batch, y_hard_neg)

            loss = (LOSS_WEIGHT_CONTRASTIVE * loss_con) + (LOSS_WEIGHT_TRIPLET * loss_tri)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    scheduler.step()
    avg_loss = total_loss / len(train_loader)



 # ----- Validation -----
    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset)
    print(f"Epoch {epoch:02d}: loss={avg_loss:.4f} | "
          f"R@1={rec['R@1']:.4f}  R@5={rec['R@5']:.4f}  "
          f"R@10={rec['R@10']:.4f}  R@50={rec['R@50']:.4f}")

    if rec['R@10'] > best_r10:
        best_r10 = rec['R@10']
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"✅ Best model saved (epoch {epoch}, R@10={best_r10:.4f})")
print(f"\n🎯 Best R@10 on validation = {best_r10:.4f}\nModel saved as {SAVE_PATH}")


# ------------------------------------------------------
# 9. Inference + Submission
# ------------------------------------------------------
print(f"Loading best model from {SAVE_PATH} for inference...")
model = ResidualPatchingTransformer(
    R_init=R.detach().clone(),
    patch_size=64, embed_dim=256, depth=3, nhead=8
).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in range(0, len(tx_test_t), 1024):
        chunk = tx_test_t[i:i+1024]
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_patching_transformer.csv", index=False)
print("\n✅ submission_patching_transformer.csv saved successfully.")


Using device: cuda
Train shapes: (125000, 1024), (25000, 1536), (125000, 1536)
Test shape: (1500, 1024)
Data preprocessed and normalized.


/tmp/ipython-input-670452080.py:200: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # Mixed precision


Computed orthogonal base R: torch.Size([1536, 1024])
Train pairs: 112500, Validation pairs: 12500

Training with Patching Transformer + Correct Hard-Negative Mining...



Epoch 1/40:   0%|          | 0/220 [00:00<?, ?it/s]/tmp/ipython-input-670452080.py:212: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 01: loss=2.2878 | R@1=0.0802  R@5=0.2088  R@10=0.2994  R@50=0.5694
✅ Best model saved (epoch 1, R@10=0.2994)


...

Epoch 36: loss=0.7007 | R@1=0.1546  R@5=0.3392  R@10=0.4404  R@50=0.6860


Epoch 37: loss=0.6981 | R@1=0.1536  R@5=0.3386  R@10=0.4396  R@50=0.6854


Epoch 38: loss=0.6946 | R@1=0.1532  R@5=0.3388  R@10=0.4400  R@50=0.6860


Epoch 39: loss=0.6947 | R@1=0.1546  R@5=0.3392  R@10=0.4404  R@50=0.6864


Epoch 40: loss=0.6928 | R@1=0.1546  R@5=0.3392  R@10=0.4402  R@50=0.6862

🎯 Best R@10 on validation = 0.4622
Model saved as patching_transformer_best.pth
Loading best model from patching_transformer_best.pth for inference...
Best model loaded.

✅ submission_patching_transformer.csv saved successfully.


# Relative Anchor Loss

In [ ]:

# ------------------------------------------------------
# 1. Device and data loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_data = np.load("train.npz")
test_data  = np.load("test.clean.npz")

tx_train = train_data["captions/embeddings"]
im_train = train_data["images/embeddings"]
tx_test  = test_data["captions/embeddings"]

repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)
print(f"Train shapes: {tx_train.shape}, {im_train.shape}, {im_train_expanded.shape}")
print(f"Test shape: {tx_test.shape}")

# ------------------------------------------------------
# 2. Data preprocessing (Centering + Normalization)
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)
tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)
print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 3. Orthogonal Procrustes base (R)
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t_unique.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh
print(f"Computed orthogonal base R: {R.shape}")

# ------------------------------------------------------
# 4. MODEL: EnhancedResidualTranslator
#    (Our best-performing model from the 0.5616 run)
# ------------------------------------------------------
class EnhancedResidualTranslator(nn.Module):
    def __init__(self, R_init, input_dim=1024, hidden_dim=2048, output_dim=1536, dropout=0.1):
        super().__init__()
        self.register_buffer("R", R_init)
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        base = x @ self.R.T
        res = self.residual(x)
        return F.normalize(base + res, p=2, dim=1)

# ------------------------------------------------------
# 5. Loss Functions (Hard-Negative + NEW Relative Loss)
# ------------------------------------------------------
TAU = 0.07 # Temperature for contrastive loss
LOSS_WEIGHT_PRIMARY = 1.0     # (Contrastive + Triplet)
LOSS_WEIGHT_RELATIVE = 0.5    # NEW loss term

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=0.2
)

def relative_anchor_loss(x_batch, y_pred, anchors_x, anchors_y):
    """
    Forces the relative similarities to anchors to be the same.
    x_batch: (B, 1024), y_pred: (B, 1536)
    anchors_x: (K, 1024), anchors_y: (K, 1536)
    """
    # (B, K)
    sim_x_rel = x_batch @ anchors_x.T
    sim_y_rel = y_pred @ anchors_y.T

    # Force the two similarity vectors to be as close as possible
    return F.mse_loss(sim_x_rel, sim_y_rel)

# ------------------------------------------------------
# 6. Validation split & Anchor Selection
# ------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)
idx_cpu = torch.randperm(N, device="cpu") # Use CPU for randperm
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_train = (train_idx // 5).to(device)
img_indices_val = (val_idx // 5).to(device)

tx_val_t, im_val_t = tx_train_t[val_idx], im_train_exp[val_idx]
tx_train_t_sub, im_train_exp_sub = tx_train_t[train_idx], im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# --- Create Anchors for the Relative Loss ---
# We use a fixed set of 1024 anchors from the validation set
NUM_ANCHORS = 1024
anchor_indices = torch.randperm(len(val_idx), device="cpu")[:NUM_ANCHORS]
anchors_x = tx_val_t[anchor_indices].detach() # (K, 1024)
anchors_y = im_val_t[anchor_indices].detach() # (K, 1536)
print(f"Created {NUM_ANCHORS} anchors for relative loss.")

# ------------------------------------------------------
# 7. Recall@K utility (Unchanged)
# ------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, repeat_factor=5, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    preds_list = []
    for i in range(0, len(tx_queries), 1024):
        chunk = tx_queries[i:i+1024]
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0) # (N_queries, 1536)

    sim = preds @ im_database.T # (N_queries, N_images)

    knn_q = torch.topk(sim, k=k_csls, dim=1).values
    mean_knn_q = knn_q.mean(1, keepdim=True) # (N_queries, 1)

    knn_d = torch.topk(sim.T, k=k_csls, dim=1).values
    mean_knn_d = knn_d.mean(1, keepdim=True).T # (1, N_images)

    csls_sim = 2 * sim - mean_knn_q - mean_knn_d # (N_queries, N_images)

    gt = query_img_indices # (N_queries,)

    top_indices = torch.argsort(csls_sim, dim=1, descending=True)

    recalls = {}
    for k in ks:
        top_k_preds = top_indices[:, :k]
        correct_in_top_k = (top_k_preds == gt.unsqueeze(1)).any(dim=1)
        recall_at_k = correct_in_top_k.float().mean().item()
        recalls[f"R@{k}"] = recall_at_k

    return recalls

# ------------------------------------------------------
# 8. Training Loop (with NEW loss)
# ------------------------------------------------------
EPOCHS = 50 # Train for longer
LR = 1e-4
SAVE_PATH = "relative_anchor_best.pth" # New save path

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

# --- Initialize NEW model ---
model = EnhancedResidualTranslator(R.detach().clone(), hidden_dim=2048).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
# Scheduler now runs for 50 epochs
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler() # Mixed precision

best_r10 = 0.0
print(f"\nTraining with Enhanced Model + Relative Anchor Loss...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            y_pred = model(x_batch)

            # --- 1. Primary Loss (Hard-Negative) ---
            sims_with_tau = y_pred @ y_batch.T / TAU
            labels = torch.arange(y_pred.size(0), device=device)
            loss_con = F.cross_entropy(sims_with_tau, labels)

            with torch.no_grad():
                sims_no_tau = y_pred @ y_batch.T
                positive_mask = (img_indices.unsqueeze(1) == img_indices.unsqueeze(0))
                sims_no_tau.masked_fill_(positive_mask, -float('inf'))
                hard_neg_idx = sims_no_tau.argmax(dim=1)

            y_hard_neg = y_batch[hard_neg_idx]
            loss_tri = triplet_loss_fn(y_pred, y_batch, y_hard_neg)

            loss_primary = (0.7 * loss_con) + (0.3 * loss_tri)

            # --- 2. NEW Relative Anchor Loss ---
            loss_rel = relative_anchor_loss(x_batch, y_pred, anchors_x, anchors_y)

            # --- 3. Combine ---
            loss = (LOSS_WEIGHT_PRIMARY * loss_primary) + (LOSS_WEIGHT_RELATIVE * loss_rel)

        scaler.scale(loss).backward()
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    scheduler.step()
    avg_loss = total_loss / len(train_loader)

    # ----- Validation -----
    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset)
    print(f"Epoch {epoch:02d}: loss={avg_loss:.4f} | "
          f"R@1={rec['R@1']:.4f}  R@5={rec['R@5']:.4f}  "
          f"R@10={rec['R@10']:.4f}  R@50={rec['R@50']:.4f}")

    if rec['R@10'] > best_r10:
        best_r10 = rec['R@10']
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"✅ Best model saved (epoch {epoch}, R@10={best_r10:.4f})")

print(f"\n🎯 Best R@10 on validation = {best_r10:.4f}\nModel saved as {SAVE_PATH}")

# ------------------------------------------------------
# 9. Inference + Submission
# ------------------------------------------------------
print(f"Loading best model from {SAVE_PATH} for inference...")
model = EnhancedResidualTranslator(R.detach().clone(), hidden_dim=2048).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in range(0, len(tx_test_t), 1024):
        chunk = tx_test_t[i:i+1024]
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_relative_anchor_loss.csv", index=False)
print("\n✅ submission_relative_anchor_loss.csv saved successfully.")


Using device: cuda
Train shapes: (125000, 1024), (25000, 1536), (125000, 1536)
Test shape: (1500, 1024)
Data preprocessed and normalized.


/tmp/ipython-input-661048062.py:173: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # Mixed precision


Computed orthogonal base R: torch.Size([1536, 1024])
Train pairs: 112500, Validation pairs: 12500
Created 1024 anchors for relative loss.

Training with Enhanced Model + Relative Anchor Loss...



Epoch 1/50:   0%|          | 0/220 [00:00<?, ?it/s]/tmp/ipython-input-661048062.py:185: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 01: loss=2.5054 | R@1=0.0822  R@5=0.2190  R@10=0.2984  R@50=0.5540
✅ Best model saved (epoch 1, R@10=0.2984)


...

Epoch 46: loss=1.2500 | R@1=0.1880  R@5=0.3894  R@10=0.4996  R@50=0.7502


Epoch 47: loss=1.2499 | R@1=0.1880  R@5=0.3892  R@10=0.4998  R@50=0.7502


Epoch 48: loss=1.2504 | R@1=0.1880  R@5=0.3892  R@10=0.4996  R@50=0.7504


Epoch 49: loss=1.2497 | R@1=0.1878  R@5=0.3892  R@10=0.4996  R@50=0.7504


Epoch 50: loss=1.2493 | R@1=0.1878  R@5=0.3892  R@10=0.4998  R@50=0.7504

🎯 Best R@10 on validation = 0.4998
Model saved as relative_anchor_best.pth
Loading best model from relative_anchor_best.pth for inference...
Best model loaded.

✅ submission_relative_anchor_loss.csv saved successfully.


#  Lightweight Transformer


In [ ]:

# ------------------------------------------------------
# 1. Device and data loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_data = np.load("train.npz")
test_data  = np.load("test.clean.npz")

tx_train = train_data["captions/embeddings"]
im_train = train_data["images/embeddings"]
tx_test  = test_data["captions/embeddings"]

repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)
print(f"Train shapes: {tx_train.shape}, {im_train.shape}, {im_train_expanded.shape}")
print(f"Test shape: {tx_test.shape}")

# ------------------------------------------------------
# 2. Data preprocessing (Centering + Normalization)
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)
tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)
print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 3. Orthogonal Procrustes base (R)
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t_unique.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh
print(f"Computed orthogonal base R: {R.shape}")

# ------------------------------------------------------
# 4. NEW MODEL: LightweightRegularizedTransformer
# ------------------------------------------------------
class LightweightRegularizedTransformer(nn.Module):
    def __init__(self, R_init,
                 input_dim=1024,
                 output_dim=1536,
                 patch_size=64,
                 embed_dim=256,
                 depth=2,  # SHALLOW: Only 2 layers
                 nhead=4,  # FEWER HEADS: 4 instead of 8
                 dropout=0.2): # HIGHER DROPOUT
        super().__init__()
        self.register_buffer("R", R_init)

        assert input_dim % patch_size == 0, "Input dim must be divisible by patch size"
        num_patches = input_dim // patch_size # 1024 // 64 = 16 patches

        # --- 1. Patching and Embedding ---
        self.patch_embed = nn.Linear(input_dim, embed_dim * num_patches)
        self.pos_embed = nn.Parameter(torch.randn(1, num_patches, embed_dim) * 0.02)

        self.num_patches = num_patches
        self.embed_dim = embed_dim
        self.dropout = nn.Dropout(dropout)

        # --- 2. Transformer Encoder ---
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=nhead,
            dim_feedforward=embed_dim * 2, # Smaller FFN
            dropout=dropout,
            activation='gelu',
            batch_first=True,
            norm_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=depth)

        # --- 3. Output Projection ---
        self.norm = nn.LayerNorm(embed_dim)
        self.output_proj = nn.Linear(embed_dim * num_patches, output_dim)

    def forward(self, x):
        # 1. Base (Orthogonal) Path
        base = x @ self.R.T # (B, 1536)

        # 2. Residual (Transformer) Path
        x_res = self.patch_embed(x)
        x_res = x_res.view(-1, self.num_patches, self.embed_dim)
        x_res = x_res + self.pos_embed
        x_res = self.dropout(x_res)
        x_res = self.transformer(x_res)
        x_res = self.norm(x_res)
        x_res = x_res.flatten(start_dim=1)
        res = self.output_proj(x_res)

        # 3. Combine and Normalize
        return F.normalize(base + res, p=2, dim=1)

# ------------------------------------------------------
# 5. Loss Functions (Our proven Hard-Negative setup)
# ------------------------------------------------------
TAU = 0.07 # Temperature for contrastive loss
LOSS_WEIGHT_CONTRASTIVE = 0.7
LOSS_WEIGHT_TRIPLET = 0.3

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=0.2
)

# ------------------------------------------------------
# 6. Validation split (Unchanged)
# ------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)
idx_cpu = torch.randperm(N, device="cpu")
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_train = (train_idx // 5).to(device)
img_indices_val = (val_idx // 5).to(device)

tx_val_t, im_val_t = tx_train_t[val_idx], im_train_exp[val_idx]
tx_train_t_sub, im_train_exp_sub = tx_train_t[train_idx], im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# ------------------------------------------------------
# 7. Recall@K utility (Unchanged)
# ------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, repeat_factor=5, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    preds_list = []
    for i in range(0, len(tx_queries), 1024):
        chunk = tx_queries[i:i+1024]
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0)

    sim = preds @ im_database.T

    knn_q = torch.topk(sim, k=k_csls, dim=1).values
    mean_knn_q = knn_q.mean(1, keepdim=True)

    knn_d = torch.topk(sim.T, k=k_csls, dim=1).values
    mean_knn_d = knn_d.mean(1, keepdim=True).T

    csls_sim = 2 * sim - mean_knn_q - mean_knn_d

    gt = query_img_indices

    top_indices = torch.argsort(csls_sim, dim=1, descending=True)

    recalls = {}
    for k in ks:
        top_k_preds = top_indices[:, :k]
        correct_in_top_k = (top_k_preds == gt.unsqueeze(1)).any(dim=1)
        recall_at_k = correct_in_top_k.float().mean().item()
        recalls[f"R@{k}"] = recall_at_k

    return recalls

# ------------------------------------------------------
# 8. Training Loop (with NEW model and HIGHER weight_decay)
# ------------------------------------------------------
EPOCHS = 40
LR = 1e-4
WEIGHT_DECAY = 1e-4 # STRONGER REGULARIZATION
SAVE_PATH = "lightweight_transformer_best.pth" # New save path

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

# --- Initialize NEW model ---
model = LightweightRegularizedTransformer(
    R_init=R.detach().clone()
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler() # Mixed precision

best_r10 = 0.0
print(f"\nTraining with Lightweight Transformer (Regularized)...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            y_pred = model(x_batch)

            sims_with_tau = y_pred @ y_batch.T / TAU
            labels = torch.arange(y_pred.size(0), device=device)
            loss_con = F.cross_entropy(sims_with_tau, labels)

            with torch.no_grad():
                sims_no_tau = y_pred @ y_batch.T
                positive_mask = (img_indices.unsqueeze(1) == img_indices.unsqueeze(0))
                sims_no_tau.masked_fill_(positive_mask, -float('inf'))
                hard_neg_idx = sims_no_tau.argmax(dim=1)

            y_hard_neg = y_batch[hard_neg_idx]
            loss_tri = triplet_loss_fn(y_pred, y_batch, y_hard_neg)

            loss = (LOSS_WEIGHT_CONTRASTIVE * loss_con) + (LOSS_WEIGHT_TRIPLET * loss_tri)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Gradient Clipping
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    scheduler.step()
    avg_loss = total_loss / len(train_loader)

    # ----- Validation -----
    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset)
    print(f"Epoch {epoch:02d}: loss={avg_loss:.4f} | "
          f"R@1={rec['R@1']:.4f}  R@5={rec['R@5']:.4f}  "
          f"R@10={rec['R@10']:.4f}  R@50={rec['R@50']:.4f}")

    if rec['R@10'] > best_r10:
        best_r10 = rec['R@10']
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"✅ Best model saved (epoch {epoch}, R@10={best_r10:.4f})")

print(f"\n🎯 Best R@10 on validation = {best_r10:.4f}\nModel saved as {SAVE_PATH}")

# ------------------------------------------------------
# 9. Inference + Submission
# ------------------------------------------------------
print(f"Loading best model from {SAVE_PATH} for inference...")
model = LightweightRegularizedTransformer(
    R_init=R.detach().clone()
).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in range(0, len(tx_test_t), 1024):
        chunk = tx_test_t[i:i+1024]
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_lightweight_transformer.csv", index=False)
print("\n✅ submission_lightweight_transformer.csv saved successfully.")

Using device: cuda
Train shapes: (125000, 1024), (25000, 1536), (125000, 1536)
Test shape: (1500, 1024)
Data preprocessed and normalized.


/tmp/ipython-input-2089763401.py:188: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # Mixed precision


Computed orthogonal base R: torch.Size([1536, 1024])
Train pairs: 112500, Validation pairs: 12500

Training with Lightweight Transformer (Regularized)...



Epoch 1/40:   0%|          | 0/220 [00:00<?, ?it/s]/tmp/ipython-input-2089763401.py:200: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 01: loss=2.8066 | R@1=0.0436  R@5=0.1308  R@10=0.1994  R@50=0.4430
✅ Best model saved (epoch 1, R@10=0.1994)


...

Epoch 36: loss=1.5962 | R@1=0.1106  R@5=0.2846  R@10=0.3898  R@50=0.6680


Epoch 37: loss=1.5952 | R@1=0.1104  R@5=0.2846  R@10=0.3900  R@50=0.6676


Epoch 38: loss=1.5963 | R@1=0.1104  R@5=0.2852  R@10=0.3902  R@50=0.6678
✅ Best model saved (epoch 38, R@10=0.3902)


Epoch 39: loss=1.5944 | R@1=0.1104  R@5=0.2854  R@10=0.3898  R@50=0.6678


Epoch 40: loss=1.5952 | R@1=0.1104  R@5=0.2852  R@10=0.3898  R@50=0.6678

🎯 Best R@10 on validation = 0.3902
Model saved as lightweight_transformer_best.pth
Loading best model from lightweight_transformer_best.pth for inference...
Best model loaded.

✅ submission_lightweight_transformer.csv saved successfully.


In [ ]:
Epoch 50: loss=1.2493 | R@1=0.1878  R@5=0.3892  R@10=0.4998  R@50=0.7504

# Pure Contrasitive Loss

#

In [ ]:
# ======================================================
#  AML Challenge — Residual-Orthogonal Translator v5
#  Fine-tuned version of pure-contrastive model
#  Adds: 0.1 Triplet term + tau=0.065 + CSLS inference
# ======================================================

import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np, pandas as pd
import os, math
from google.colab import drive

# ------------------------------------------------------
# 0. Setup Google Drive
# ------------------------------------------------------
print("Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)

BASE_DIR = "/content/drive/MyDrive/AML Challenge"
os.makedirs(BASE_DIR, exist_ok=True)
os.chdir(BASE_DIR)
print(f"Current working directory: {os.getcwd()}")

# ------------------------------------------------------
# 1. Device + Data Loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_data = np.load("train.npz")
test_data  = np.load("test.clean.npz")

tx_train = train_data["captions/embeddings"]
im_train = train_data["images/embeddings"]
tx_test  = test_data["captions/embeddings"]

repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)

# ------------------------------------------------------
# 2. Preprocessing
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test, dtype=torch.float32, device=device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)

tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)

print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 3. Orthogonal Procrustes base
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t_unique.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh
print(f"Computed orthogonal base R: {R.shape}")

# ------------------------------------------------------
# 4. Model
# ------------------------------------------------------
class ResidualTranslator(nn.Module):
    def __init__(self, R_init, input_dim=1024, hidden_dim=1024, output_dim=1536):
        super().__init__()
        self.register_buffer("R", R_init)
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, output_dim),
        )
    def forward(self, x):
        base = x @ self.R.T
        res  = self.residual(x)
        return F.normalize(base + res, p=2, dim=1)

# ------------------------------------------------------
# 5. Loss functions
# ------------------------------------------------------
TAU = 0.065

def pure_contrastive_loss(y_pred, y_batch, img_indices):
    sims = y_pred @ y_batch.T / TAU
    positive_mask = (img_indices.unsqueeze(1) == img_indices.unsqueeze(0)).float()
    log_probs = F.log_softmax(sims, dim=1) * positive_mask
    num_pos = torch.clamp(positive_mask.sum(1), min=1.0)
    loss = -log_probs.sum(1) / num_pos
    return loss.mean()

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=0.15
)

# ------------------------------------------------------
# 6. Validation split
# ------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)
idx_cpu = torch.randperm(N, device="cpu")
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_train = (train_idx // 5).to(device)
img_indices_val = (val_idx // 5).to(device)

tx_val_t, im_val_t = tx_train_t[val_idx], im_train_exp[val_idx]
tx_train_t_sub, im_train_exp_sub = tx_train_t[train_idx], im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# ------------------------------------------------------
# 7. Recall@K
# ------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, ks=(1,5,10,50), k_csls=10):
    model.eval()
    preds = []
    for i in range(0, len(tx_queries), 1024):
        preds.append(model(tx_queries[i:i+1024].to(device)))
    preds = torch.cat(preds, dim=0)
    im_database = im_database.to(device)
    sim = preds @ im_database.T

    knn_q = torch.topk(sim, k=k_csls, dim=1).values.mean(1, keepdim=True)
    knn_d = torch.topk(sim.T, k=k_csls, dim=1).values.mean(1, keepdim=True).T
    csls_sim = 2 * sim - knn_q - knn_d

    gt = query_img_indices.to(device)
    ranks = torch.argsort(csls_sim, dim=1, descending=True)
    recalls = {}
    for k in ks:
        match = (ranks[:, :k] == gt.unsqueeze(1)).any(dim=1)
        recalls[f"R@{k}"] = match.float().mean().item()
    return recalls

# ------------------------------------------------------
# 8. Training
# ------------------------------------------------------
EPOCHS = 45
LR = 1e-4
SAVE_PATH = "residual_v5_best.pth"

val_subset_queries = tx_val_t[:5000]
val_subset_indices = img_indices_val[:5000]
val_db_subset = im_train_t_unique

model = ResidualTranslator(R.detach().clone()).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler()

best_r10 = 0.0
print("\nTraining Residual-Orthogonal Translator v5 (Contrastive + small Triplet)...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    for x_batch, y_batch, img_idx in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            y_pred = model(x_batch)
            loss_con = pure_contrastive_loss(y_pred, y_batch, img_idx)
            # create negatives by random permutation
            neg_idx = torch.randperm(y_batch.size(0), device=device)
            y_neg = y_batch[neg_idx]
            loss_tri = triplet_loss_fn(y_pred, y_batch, y_neg)
            loss = 0.9 * loss_con + 0.1 * loss_tri
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
    scheduler.step()

    avg_loss = total_loss / len(train_loader)
    rec = recall_at_k(model, val_subset_queries, val_db_subset, val_subset_indices)
    print(f"Epoch {epoch:02d}: loss={avg_loss:.4f} | "
          f"R@1={rec['R@1']:.4f} R@5={rec['R@5']:.4f} R@10={rec['R@10']:.4f} R@50={rec['R@50']:.4f}")
    if rec['R@10'] > best_r10:
        best_r10 = rec['R@10']
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"✅ Best model updated (epoch {epoch}, R@10={best_r10:.4f})")

print(f"\n🎯 Best validation R@10 = {best_r10:.4f}\nModel saved as {SAVE_PATH}")

# ------------------------------------------------------
# 9. Inference + CSLS Submission
# ------------------------------------------------------
@torch.no_grad()
def apply_csls(preds, im_base, k=10):
    preds = F.normalize(torch.tensor(preds, device=device), p=2, dim=1)
    im_base = F.normalize(im_base, p=2, dim=1)
    sim = preds @ im_base.T
    knn_q = torch.topk(sim, k=k, dim=1).values.mean(1, keepdim=True)
    knn_d = torch.topk(sim, k=k, dim=0).values.mean(0, keepdim=True)
    return (2 * sim - knn_q - knn_d).cpu().numpy()

print(f"\nLoading {SAVE_PATH} for inference...")
model = ResidualTranslator(R.detach().clone()).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_all = []
    for i in range(0, len(tx_test_t), 1024):
        preds_all.append(model(tx_test_t[i:i+1024].to(device)))
    preds = torch.cat(preds_all, dim=0).cpu().numpy()

csls_sim = apply_csls(preds, im_train_t_unique)
test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_residual_v5.csv", index=False)
print("\n✅ submission_residual_v5.csv saved successfully.")


Mounting Google Drive...


Exception ignored in: <function NpzFile.__del__ at 0x7e43f5d17a60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/numpy/lib/_npyio_impl.py", line 226, in __del__
    self.close()
  File "/usr/local/lib/python3.12/dist-packages/numpy/lib/_npyio_impl.py", line 221, in close
    self.fid.close()
OSError: [Errno 107] Transport endpoint is not connected
Exception ignored in: <function NpzFile.__del__ at 0x7e43f5d17a60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/numpy/lib/_npyio_impl.py", line 226, in __del__
    self.close()
  File "/usr/local/lib/python3.12/dist-packages/numpy/lib/_npyio_impl.py", line 221, in close
    self.fid.close()
OSError: [Errno 107] Transport endpoint is not connected


Mounted at /content/drive
Current working directory: /content/drive/MyDrive/AML Challenge
Using device: cuda
Data preprocessed and normalized.


/tmp/ipython-input-853231009.py:161: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Computed orthogonal base R: torch.Size([1536, 1024])
Train pairs: 112500, Validation pairs: 12500

Training Residual-Orthogonal Translator v5 (Contrastive + small Triplet)...



...

Epoch 41: loss=1.7804 | R@1=0.1968 R@5=0.4116 R@10=0.5144 R@50=0.7468
✅ Best model updated (epoch 41, R@10=0.5144)


Epoch 42: loss=1.7806 | R@1=0.1966 R@5=0.4116 R@10=0.5144 R@50=0.7470


Epoch 43: loss=1.7801 | R@1=0.1968 R@5=0.4116 R@10=0.5144 R@50=0.7470


Epoch 44: loss=1.7804 | R@1=0.1966 R@5=0.4116 R@10=0.5144 R@50=0.7470


Epoch 45: loss=1.7789 | R@1=0.1966 R@5=0.4116 R@10=0.5144 R@50=0.7470

🎯 Best validation R@10 = 0.5144
Model saved as residual_v5_best.pth

Loading residual_v5_best.pth for inference...
Best model loaded.

✅ submission_residual_v5.csv saved successfully.


# 120 Epoches

In [ ]:

# ------------------------------------------------------
# 1. Device and data loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_data = np.load("train.npz")
test_data  = np.load("test.clean.npz")

tx_train = train_data["captions/embeddings"]
im_train = train_data["images/embeddings"]
tx_test  = test_data["captions/embeddings"]

repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)
print(f"Train shapes: {tx_train.shape}, {im_train.shape}, {im_train_expanded.shape}")
print(f"Test shape: {tx_test.shape}")

# ------------------------------------------------------
# 2. Data preprocessing (Centering + Normalization)
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)
tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)
print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 3. Orthogonal Procrustes base (R)
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t_unique.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh
print(f"Computed orthogonal base R: {R.shape}")

# ------------------------------------------------------
# 4. MODEL: ResidualTranslator (Slightly Enhanced)
# ------------------------------------------------------
class ResidualTranslator(nn.Module):
    # Our 0.815 winner used hidden_dim=1024
    # Our 0.796 loser used hidden_dim=2048
    # Let's try the middle ground: 1536
    def __init__(self, R_init, input_dim=1024, hidden_dim=1536, output_dim=1536, dropout=0.1):
        super().__init__()
        self.register_buffer("R", R_init)
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.LayerNorm(hidden_dim), # Add LayerNorm for stability
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        base = x @ self.R.T
        res = self.residual(x)
        return F.normalize(base + res, p=2, dim=1)

# ------------------------------------------------------
# 5. Loss Functions (Our proven Hard-Negative setup)
# ------------------------------------------------------
TAU = 0.07 # Temperature for contrastive loss
LOSS_WEIGHT_CONTRASTIVE = 0.7
LOSS_WEIGHT_TRIPLET = 0.3

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=0.2
)

# ------------------------------------------------------
# 6. Validation split (Unchanged)
# ------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)
idx_cpu = torch.randperm(N, device="cpu")
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_train = (train_idx // 5).to(device)
img_indices_val = (val_idx // 5).to(device)

tx_val_t, im_val_t = tx_train_t[val_idx], im_train_exp[val_idx]
tx_train_t_sub, im_train_exp_sub = tx_train_t[train_idx], im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# ------------------------------------------------------
# 7. Recall@K utility (Unchanged)
# ------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, repeat_factor=5, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    preds_list = []
    for i in range(0, len(tx_queries), 1024):
        chunk = tx_queries[i:i+1024].to(device) # Ensure chunk is on device
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0)

    im_database = im_database.to(device) # Ensure database is on device
    query_img_indices = query_img_indices.to(device) # Ensure indices are on device

    sim = preds @ im_database.T

    knn_q = torch.topk(sim, k=k_csls, dim=1).values
    mean_knn_q = knn_q.mean(1, keepdim=True)

    knn_d = torch.topk(sim.T, k=k_csls, dim=1).values
    mean_knn_d = knn_d.mean(1, keepdim=True).T

    csls_sim = 2 * sim - mean_knn_q - mean_knn_d

    gt = query_img_indices

    top_indices = torch.argsort(csls_sim, dim=1, descending=True)

    recalls = {}
    for k in ks:
        top_k_preds = top_indices[:, :k]
        correct_in_top_k = (top_k_preds == gt.unsqueeze(1)).any(dim=1)
        recall_at_k = correct_in_top_k.float().mean().item()
        recalls[f"R@{k}"] = recall_at_k

    return recalls

# ------------------------------------------------------
# 8. Training Loop (LONG HAUL)
# ------------------------------------------------------
EPOCHS = 120 # Train for much longer
LR = 1e-4
WEIGHT_DECAY = 5e-5 # Standard regularization
SAVE_PATH = "final_push_best.pth" # New save path

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

# --- Initialize model with modest capacity boost ---
model = ResidualTranslator(
    R_init=R.detach().clone(),
    hidden_dim=1536 # 1024 was good, 2048 was bad. Let's try 1536.
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Schedule the LR over the *entire* long run
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler() # Mixed precision

best_r10 = 0.0
print(f"\nTraining Final Push Model (120 Epochs, hidden_dim=1536)...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            y_pred = model(x_batch)

            # --- Our proven winning loss combo ---
            sims_with_tau = y_pred @ y_batch.T / TAU
            labels = torch.arange(y_pred.size(0), device=device)
            loss_con = F.cross_entropy(sims_with_tau, labels)

            with torch.no_grad():
                sims_no_tau = y_pred @ y_batch.T
                positive_mask = (img_indices.unsqueeze(1) == img_indices.unsqueeze(0))
                sims_no_tau.masked_fill_(positive_mask, -float('inf'))
                hard_neg_idx = sims_no_tau.argmax(dim=1)

            y_hard_neg = y_batch[hard_neg_idx]
            loss_tri = triplet_loss_fn(y_pred, y_batch, y_hard_neg)

            loss = (LOSS_WEIGHT_CONTRASTIVE * loss_con) + (LOSS_WEIGHT_TRIPLET * loss_tri)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Gradient Clipping
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    scheduler.step()
    avg_loss = total_loss / len(train_loader)

    # ----- Validation -----
    # Only validate every 2 epochs to save time
    if epoch % 2 == 0 or epoch == EPOCHS:
        rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset)
        print(f"Epoch {epoch:03d}: loss={avg_loss:.4f} | "
              f"R@1={rec['R@1']:.4f}  R@5={rec['R@5']:.4f}  "
              f"R@10={rec['R@10']:.4f}  R@50={rec['R@50']:.4f}")

        if rec['R@10'] > best_r10:
            best_r10 = rec['R@10']
            torch.save(model.state_dict(), SAVE_PATH)
            print(f"✅ Best model saved (epoch {epoch}, R@10={best_r10:.4f})")

print(f"\n🎯 Best R@10 on validation = {best_r10:.4f}\nModel saved as {SAVE_PATH}")

# ------------------------------------------------------
# 9. Inference + Submission
# ------------------------------------------------------
print(f"Loading best model from {SAVE_PATH} for inference...")
model = ResidualTranslator(
    R_init=R.detach().clone(),
    hidden_dim=1536
).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in range(0, len(tx_test_t), 1024):
        chunk = tx_test_t[i:i+1024].to(device) # Ensure chunk is on device
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_final_push.csv", index=False)
print("\n✅ submission_final_push.csv saved successfully.")


Using device: cuda
Train shapes: (125000, 1024), (25000, 1536), (125000, 1536)
Test shape: (1500, 1024)
Data preprocessed and normalized.


/tmp/ipython-input-1287973950.py:157: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # Mixed precision


Computed orthogonal base R: torch.Size([1536, 1024])
Train pairs: 112500, Validation pairs: 12500

Training Final Push Model (120 Epochs, hidden_dim=1536)...



Epoch 1/120:   0%|          | 0/220 [00:00<?, ?it/s]/tmp/ipython-input-1287973950.py:169: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 002: loss=2.0280 | R@1=0.1054  R@5=0.2632  R@10=0.3570  R@50=0.6082
✅ Best model saved (epoch 2, R@10=0.3570)


...

Epoch 112: loss=1.4644 | R@1=0.1664  R@5=0.3692  R@10=0.4746  R@50=0.7212


Epoch 114: loss=1.4635 | R@1=0.1664  R@5=0.3692  R@10=0.4746  R@50=0.7214


Epoch 116: loss=1.4647 | R@1=0.1664  R@5=0.3692  R@10=0.4748  R@50=0.7212


Epoch 118: loss=1.4643 | R@1=0.1664  R@5=0.3692  R@10=0.4746  R@50=0.7212


Epoch 120: loss=1.4635 | R@1=0.1664  R@5=0.3692  R@10=0.4748  R@50=0.7212

🎯 Best R@10 on validation = 0.4754
Model saved as final_push_best.pth
Loading best model from final_push_best.pth for inference...
Best model loaded.

✅ submission_final_push.csv saved successfully.


# Fine TUning the original Residual model
# Highest score (0.82388) as per 10/30/2025

In [ ]:
# ----------------------------------------------------
# 1. Device and data loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_data = np.load("train.npz")
test_data  = np.load("test.clean.npz")

tx_train = train_data["captions/embeddings"]
im_train = train_data["images/embeddings"]
tx_test  = test_data["captions/embeddings"]

repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)
print(f"Train shapes: {tx_train.shape}, {im_train.shape}, {im_train_expanded.shape}")
print(f"Test shape: {tx_test.shape}")

# ------------------------------------------------------
# 2. Data preprocessing (Centering + Normalization)
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)
tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)
print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 3. Orthogonal Procrustes base (R)
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t_unique.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh
print(f"Computed orthogonal base R: {R.shape}")

# ------------------------------------------------------
# 4. MODEL: ResidualTranslator (Our 0.815 Kaggle Winner)
# ------------------------------------------------------
class ResidualTranslator(nn.Module):
    def __init__(self, R_init, input_dim=1024, hidden_dim=1024, output_dim=1536):
        super().__init__()
        self.register_buffer("R", R_init)
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        base = x @ self.R.T
        res = self.residual(x)
        return F.normalize(base + res, p=2, dim=1)

# ------------------------------------------------------
# 5. Loss Functions (Proven Hard-Negative, Tuned Hyperparams)
# ------------------------------------------------------
# --- HYPERPARAMETER TUNE ---
TAU = 0.05       # Lower temperature (was 0.07) -> Stricter contrastive loss
MARGIN = 0.25    # Higher margin (was 0.2) -> Stricter triplet loss
# -------------------------

LOSS_WEIGHT_CONTRASTIVE = 0.7
LOSS_WEIGHT_TRIPLET = 0.3

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=MARGIN
)

# ------------------------------------------------------
# 6. Validation split (Unchanged)
# ------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)
idx_cpu = torch.randperm(N, device="cpu")
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_train = (train_idx // 5).to(device)
img_indices_val = (val_idx // 5).to(device)

tx_val_t, im_val_t = tx_train_t[val_idx], im_train_exp[val_idx]
tx_train_t_sub, im_train_exp_sub = tx_train_t[train_idx], im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# ------------------------------------------------------
# 7. Recall@K utility (Unchanged)
# ------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, repeat_factor=5, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    preds_list = []
    for i in range(0, len(tx_queries), 1024):
        chunk = tx_queries[i:i+1024].to(device) # Ensure chunk is on device
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0)

    im_database = im_database.to(device) # Ensure database is on device
    query_img_indices = query_img_indices.to(device) # Ensure indices are on device

    sim = preds @ im_database.T

    knn_q = torch.topk(sim, k=k_csls, dim=1).values
    mean_knn_q = knn_q.mean(1, keepdim=True)

    knn_d = torch.topk(sim.T, k=k_csls, dim=1).values
    mean_knn_d = knn_d.mean(1, keepdim=True).T

    csls_sim = 2 * sim - mean_knn_q - mean_knn_d

    gt = query_img_indices

    top_indices = torch.argsort(csls_sim, dim=1, descending=True)

    recalls = {}
    for k in ks:
        top_k_preds = top_indices[:, :k]
        correct_in_top_k = (top_k_preds == gt.unsqueeze(1)).any(dim=1)
        recall_at_k = correct_in_top_k.float().mean().item()
        recalls[f"R@{k}"] = recall_at_k

    return recalls

# ------------------------------------------------------
# 8. Training Loop (Original 40 epochs)
# ------------------------------------------------------
EPOCHS = 40 # Back to our 0.815 winner's epoch count
LR = 1e-4
WEIGHT_DECAY = 5e-5 # Standard regularization
SAVE_PATH = "hyperparam_tune_best.pth" # New save path

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

# --- Initialize WINNING model ---
model = ResidualTranslator(
    R_init=R.detach().clone(),
    input_dim=1024,
    hidden_dim=1024, # Our 0.815 winner's capacity
    output_dim=1536
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler() # Mixed precision

best_r10 = 0.0
print(f"\nTraining with Tuned Hyperparameters (Tau={TAU}, Margin={MARGIN})...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            y_pred = model(x_batch)

            # --- Our proven winning loss combo ---
            sims_with_tau = y_pred @ y_batch.T / TAU # Using new TAU
            labels = torch.arange(y_pred.size(0), device=device)
            loss_con = F.cross_entropy(sims_with_tau, labels)

            with torch.no_grad():
                sims_no_tau = y_pred @ y_batch.T
                positive_mask = (img_indices.unsqueeze(1) == img_indices.unsqueeze(0))
                sims_no_tau.masked_fill_(positive_mask, -float('inf'))
                hard_neg_idx = sims_no_tau.argmax(dim=1)

            y_hard_neg = y_batch[hard_neg_idx]
            loss_tri = triplet_loss_fn(y_pred, y_batch, y_hard_neg) # Using new MARGIN

            loss = (LOSS_WEIGHT_CONTRASTIVE * loss_con) + (LOSS_WEIGHT_TRIPLET * loss_tri)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Gradient Clipping
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    scheduler.step()
    avg_loss = total_loss / len(train_loader)

    # ----- Validation -----
    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset)
    print(f"Epoch {epoch:02d}: loss={avg_loss:.4f} | "
          f"R@1={rec['R@1']:.4f}  R@5={rec['R@5']:.4f}  "
          f"R@10={rec['R@10']:.4f}  R@50={rec['R@50']:.4f}")

    if rec['R@10'] > best_r10:
        best_r10 = rec['R@10']
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"✅ Best model saved (epoch {epoch}, R@10={best_r10:.4f})")

print(f"\n🎯 Best R@10 on validation = {best_r10:.4f}\nModel saved as {SAVE_PATH}")

# ------------------------------------------------------
# 9. Inference + Submission
# ------------------------------------------------------
print(f"Loading best model from {SAVE_PATH} for inference...")
model = ResidualTranslator(
    R_init=R.detach().clone(),
    input_dim=1024,
    hidden_dim=1024,
    output_dim=1536
).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in range(0, len(tx_test_t), 1024):
        chunk = tx_test_t[i:i+1024].to(device) # Ensure chunk is on device
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_hyperparam_tune.csv", index=False)
print("\n✅ submission_hyperparam_tune.csv saved successfully.")


Using device: cuda
Train shapes: (125000, 1024), (25000, 1536), (125000, 1536)
Test shape: (1500, 1024)
Data preprocessed and normalized.


/tmp/ipython-input-3346032053.py:157: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # Mixed precision


Computed orthogonal base R: torch.Size([1536, 1024])
Train pairs: 112500, Validation pairs: 12500

Training with Tuned Hyperparameters (Tau=0.05, Margin=0.25)...



Epoch 1/40:   0%|          | 0/220 [00:00<?, ?it/s]/tmp/ipython-input-3346032053.py:169: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 01: loss=1.5915 | R@1=0.1660  R@5=0.3586  R@10=0.4570  R@50=0.6866
✅ Best model saved (epoch 1, R@10=0.4570)


...

Epoch 36: loss=1.1786 | R@1=0.2220  R@5=0.4416  R@10=0.5454  R@50=0.7732


Epoch 37: loss=1.1779 | R@1=0.2220  R@5=0.4418  R@10=0.5452  R@50=0.7738


Epoch 38: loss=1.1766 | R@1=0.2220  R@5=0.4416  R@10=0.5450  R@50=0.7736


Epoch 39: loss=1.1772 | R@1=0.2218  R@5=0.4416  R@10=0.5452  R@50=0.7734


Epoch 40: loss=1.1783 | R@1=0.2218  R@5=0.4416  R@10=0.5452  R@50=0.7734

🎯 Best R@10 on validation = 0.5454
Model saved as hyperparam_tune_best.pth
Loading best model from hyperparam_tune_best.pth for inference...
Best model loaded.

✅ submission_hyperparam_tune.csv saved successfully.


## Fine tunning again

In [ ]:

# ------------------------------------------------------
# 1. Device and data loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_data = np.load("train.npz")
test_data  = np.load("test.clean.npz")

tx_train = train_data["captions/embeddings"]
im_train = train_data["images/embeddings"]
tx_test  = test_data["captions/embeddings"]

repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)
print(f"Train shapes: {tx_train.shape}, {im_train.shape}, {im_train_expanded.shape}")
print(f"Test shape: {tx_test.shape}")

# ------------------------------------------------------
# 2. Data preprocessing (Centering + Normalization)
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)
tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)
print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 3. Orthogonal Procrustes base (R)
# ------------------------------------------------------
tx_centroids = tx_train_t.view(-1, 5, tx_train_t.shape[1]).mean(dim=1)
M = im_train_t_unique.T @ tx_centroids
U, S, Vh = torch.linalg.svd(M, full_matrices=False)
R = U @ Vh
print(f"Computed orthogonal base R: {R.shape}")

# ------------------------------------------------------
# 4. MODEL: ResidualTranslator (Our 0.82388 Kaggle Winner)
# ------------------------------------------------------
class ResidualTranslator(nn.Module):
    def __init__(self, R_init, input_dim=1024, hidden_dim=1024, output_dim=1536):
        super().__init__()
        self.register_buffer("R", R_init)
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        base = x @ self.R.T
        res = self.residual(x)
        return F.normalize(base + res, p=2, dim=1)

# ------------------------------------------------------
# 5. Loss Functions (Our 0.82388 Winning Hyperparams)
# ------------------------------------------------------
TAU = 0.05       # Our winning stricter temperature
MARGIN = 0.25    # Our winning stricter margin
LOSS_WEIGHT_CONTRASTIVE = 0.7
LOSS_WEIGHT_TRIPLET = 0.3

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=MARGIN
)

# ------------------------------------------------------
# 6. Validation split (Unchanged)
# ------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)
idx_cpu = torch.randperm(N, device="cpu")
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_train = (train_idx // 5).to(device)
img_indices_val = (val_idx // 5).to(device)

tx_val_t, im_val_t = tx_train_t[val_idx], im_train_exp[val_idx]
tx_train_t_sub, im_train_exp_sub = tx_train_t[train_idx], im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# ------------------------------------------------------
# 7. Recall@K utility (Unchanged)
# ------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, repeat_factor=5, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    preds_list = []
    for i in range(0, len(tx_queries), 1024):
        chunk = tx_queries[i:i+1024].to(device) # Ensure chunk is on device
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0)

    im_database = im_database.to(device) # Ensure database is on device
    query_img_indices = query_img_indices.to(device) # Ensure indices are on device

    sim = preds @ im_database.T

    knn_q = torch.topk(sim, k=k_csls, dim=1).values
    mean_knn_q = knn_q.mean(1, keepdim=True)

    knn_d = torch.topk(sim.T, k=k_csls, dim=1).values
    mean_knn_d = knn_d.mean(1, keepdim=True).T

    csls_sim = 2 * sim - mean_knn_q - mean_knn_d

    gt = query_img_indices

    top_indices = torch.argsort(csls_sim, dim=1, descending=True)

    recalls = {}
    for k in ks:
        top_k_preds = top_indices[:, :k]
        correct_in_top_k = (top_k_preds == gt.unsqueeze(1)).any(dim=1)
        recall_at_k = correct_in_top_k.float().mean().item()
        recalls[f"R@{k}"] = recall_at_k

    return recalls

# ------------------------------------------------------
# 8. Training Loop (THE "MARATHON" RUN)
# ------------------------------------------------------
EPOCHS = 150 # Let's give it a truly long run
LR = 1e-4
WEIGHT_DECAY = 5e-5
SAVE_PATH = "marathon_run_best.pth" # New save path

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

# --- Initialize WINNING model ---
model = ResidualTranslator(
    R_init=R.detach().clone(),
    input_dim=1024,
    hidden_dim=1024, # Our 0.82388 winner's capacity
    output_dim=1536
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Schedule the LR over the *entire* marathon
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler() # Mixed precision

best_r10 = 0.0
# Load the weights from our 0.82388 winner to resume training
try:
    model.load_state_dict(torch.load("hyperparam_tune_best.pth"))
    best_r10 = 0.5454 # Start from our previous best
    print("✅ Loaded weights from 'hyperparam_tune_best.pth'. Resuming training.")
except Exception as e:
    print(f"Could not load previous weights, starting from scratch. Error: {e}")

print(f"\nTraining Final Marathon (Epochs: {EPOCHS}, Tau={TAU}, Margin={MARGIN})...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            y_pred = model(x_batch)

            # --- Our proven winning loss combo ---
            sims_with_tau = y_pred @ y_batch.T / TAU # Using new TAU
            labels = torch.arange(y_pred.size(0), device=device)
            loss_con = F.cross_entropy(sims_with_tau, labels)

            with torch.no_grad():
                sims_no_tau = y_pred @ y_batch.T
                positive_mask = (img_indices.unsqueeze(1) == img_indices.unsqueeze(0))
                sims_no_tau.masked_fill_(positive_mask, -float('inf'))
                hard_neg_idx = sims_no_tau.argmax(dim=1)

            y_hard_neg = y_batch[hard_neg_idx]
            loss_tri = triplet_loss_fn(y_pred, y_batch, y_hard_neg) # Using new MARGIN

            loss = (LOSS_WEIGHT_CONTRASTIVE * loss_con) + (LOSS_WEIGHT_TRIPLET * loss_tri)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Gradient Clipping
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    scheduler.step()
    avg_loss = total_loss / len(train_loader)

    # ----- Validation -----
    # Only validate every 2 epochs to save time
    if epoch % 2 == 0 or epoch == EPOCHS:
        rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset)
        print(f"Epoch {epoch:03d}: loss={avg_loss:.4f} | "
              f"R@1={rec['R@1']:.4f}  R@5={rec['R@5']:.4f}  "
              f"R@10={rec['R@10']:.4f}  R@50={rec['R@50']:.4f}")

        if rec['R@10'] > best_r10:
            best_r10 = rec['R@10']
            torch.save(model.state_dict(), SAVE_PATH)
            print(f"✅ Best model saved (epoch {epoch}, R@10={best_r10:.4f})")

print(f"\n🎯 Best R@10 on validation = {best_r10:.4f}\nModel saved as {SAVE_PATH}")

# ------------------------------------------------------
# 9. Inference + Submission
# ------------------------------------------------------
print(f"Loading best model from {SAVE_PATH} for inference...")
model = ResidualTranslator(
    R_init=R.detach().clone(),
    input_dim=1024,
    hidden_dim=1024,
    output_dim=1536
).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in range(0, len(tx_text_t), 1024):
        chunk = tx_test_t[i:i+1024].to(device) # Ensure chunk is on device
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_marathon_run.csv", index=False)
print("\n✅ submission_marathon_run.csv saved successfully.")


Using device: cuda
Train shapes: (125000, 1024), (25000, 1536), (125000, 1536)
Test shape: (1500, 1024)
Data preprocessed and normalized.


/tmp/ipython-input-2490564187.py:155: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # Mixed precision


Computed orthogonal base R: torch.Size([1536, 1024])
Train pairs: 112500, Validation pairs: 12500
✅ Loaded weights from 'hyperparam_tune_best.pth'. Resuming training.

Training Final Marathon (Epochs: 150, Tau=0.05, Margin=0.25)...



Epoch 1/150:   0%|          | 0/220 [00:00<?, ?it/s]/tmp/ipython-input-2490564187.py:175: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 002: loss=1.1780 | R@1=0.2324  R@5=0.4612  R@10=0.5588  R@50=0.7840
✅ Best model saved (epoch 2, R@10=0.5588)


...

Epoch 144: loss=1.0582 | R@1=0.2390  R@5=0.4644  R@10=0.5672  R@50=0.7914


Epoch 146: loss=1.0589 | R@1=0.2392  R@5=0.4648  R@10=0.5668  R@50=0.7914


Epoch 148: loss=1.0584 | R@1=0.2392  R@5=0.4648  R@10=0.5668  R@50=0.7914


Epoch 150: loss=1.0585 | R@1=0.2392  R@5=0.4648  R@10=0.5668  R@50=0.7914

🎯 Best R@10 on validation = 0.5672
Model saved as marathon_run_best.pth
Loading best model from marathon_run_best.pth for inference...
Best model loaded.


NameError: name 'tx_text_t' is not defined

In [ ]:

# ------------------------------------------------------
# 1. Device and data loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

test_data  = np.load("test.clean.npz")
tx_test  = test_data["captions/embeddings"]

# --- We only need the *mean* from the training set for normalization ---
# Load a small chunk to save RAM
train_data_captions = np.load("train.npz")["captions/embeddings"]
tx_mean = torch.as_tensor(train_data_captions.mean(0, keepdims=True), dtype=torch.float32, device=device)

# ------------------------------------------------------
# 2. Define the Model (MUST be identical to the one trained)
# ------------------------------------------------------
class ResidualTranslator(nn.Module):
    def __init__(self, R_init_shape, input_dim=1024, hidden_dim=1024, output_dim=1536):
        super().__init__()
        # We don't need the R_init, just a buffer of the right shape
        self.register_buffer("R", torch.zeros(R_init_shape))
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        base = x @ self.R.T
        res = self.residual(x)
        return F.normalize(base + res, p=2, dim=1)

# ------------------------------------------------------
# 3. Load Data and Model
# ------------------------------------------------------
CHECKPOINT_PATH = "marathon_run_best.pth"
SUBMISSION_PATH = "submission_marathon_run.csv"

# --- Prepare test data ---
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)
print("Test data loaded and normalized.")

# --- Load the saved model ---
print(f"Loading best model from {CHECKPOINT_PATH} for inference...")
# We must pass a dummy R_init_shape. We know it's (1536, 1024)
model = ResidualTranslator(
    R_init_shape=(1536, 1024),
    input_dim=1024,
    hidden_dim=1024,
    output_dim=1536
).to(device)

model.load_state_dict(torch.load(CHECKPOINT_PATH))
model.eval()
print("Best model loaded.")

# ------------------------------------------------------
# 4. Inference + Submission (Corrected)
# ------------------------------------------------------
with torch.no_grad():
    preds_list = []
    # --- THIS IS THE FIX: tx_test_t (not tx_text_t) ---
    for i in range(0, len(tx_test_t), 1024):
        chunk = tx_test_t[i:i+1024].to(device)
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()
print("Inference complete.")

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv(SUBMISSION_PATH, index=False)
print(f"\n✅ {SUBMISSION_PATH} saved successfully.")


Using device: cuda
Test data loaded and normalized.
Loading best model from marathon_run_best.pth for inference...
Best model loaded.
Inference complete.

✅ submission_marathon_run.csv saved successfully.


# Previous 2 blocks belongs to one fine-tunned experiment

# New Fine-Tunned experiment
# got score as 0.83413

In [ ]:
# ------------------------------------------------------
# 1. Device and data loading
# ------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

train_data = np.load("train.npz")
test_data  = np.load("test.clean.npz")

tx_train = train_data["captions/embeddings"]
im_train = train_data["images/embeddings"]
tx_test  = test_data["captions/embeddings"]

repeat_factor = len(tx_train) // len(im_train)
im_train_expanded = np.repeat(im_train, repeat_factor, axis=0)
print(f"Train shapes: {tx_train.shape}, {im_train.shape}, {im_train_expanded.shape}")
print(f"Test shape: {tx_test.shape}")

# ------------------------------------------------------
# 2. Data preprocessing (Centering + Normalization)
# ------------------------------------------------------
tx_train_t = torch.as_tensor(tx_train, dtype=torch.float32, device=device)
im_train_t_unique = torch.as_tensor(im_train, dtype=torch.float32, device=device)
tx_test_t  = torch.as_tensor(tx_test,  dtype=torch.float32, device=device)

tx_mean = tx_train_t.mean(0, keepdim=True)
im_mean = im_train_t_unique.mean(0, keepdim=True)
tx_train_t = F.normalize(tx_train_t - tx_mean, p=2, dim=1)
im_train_t_unique = F.normalize(im_train_t_unique - im_mean, p=2, dim=1)
tx_test_t = F.normalize(tx_test_t - tx_mean, p=2, dim=1)

im_train_exp = torch.as_tensor(im_train_expanded, dtype=torch.float32, device=device)
im_train_exp = F.normalize(im_train_exp - im_mean, p=2, dim=1)
print("Data preprocessed and normalized.")

# ------------------------------------------------------
# 3. Orthogonal Procrustes base (R)
# ------------------------------------------------------
# We only need the shape to initialize the model class
R_SHAPE = (1536, 1024)
print(f"Base R shape: {R_SHAPE}")

# ------------------------------------------------------
# 4. MODEL: ResidualTranslator (Our 0.83359 Kaggle Winner)
# ------------------------------------------------------
class ResidualTranslator(nn.Module):
    def __init__(self, R_init_shape, input_dim=1024, hidden_dim=1024, output_dim=1536):
        super().__init__()
        self.register_buffer("R", torch.zeros(R_init_shape))
        self.residual = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        base = x @ self.R.T
        res = self.residual(x)
        return F.normalize(base + res, p=2, dim=1)

# ------------------------------------------------------
# 5. Loss Functions (Our Winning Hyperparams)
# ------------------------------------------------------
TAU = 0.05       # Our winning stricter temperature
MARGIN = 0.25    # Our winning stricter margin
LOSS_WEIGHT_CONTRASTIVE = 0.7
LOSS_WEIGHT_TRIPLET = 0.3

triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1 - F.cosine_similarity(x, y),
    margin=MARGIN
)

# ------------------------------------------------------
# 6. Validation split (Unchanged)
# ------------------------------------------------------
N = len(tx_train_t)
val_size = int(0.1 * N)
idx_cpu = torch.randperm(N, device="cpu")
val_idx, train_idx = idx_cpu[:val_size], idx_cpu[val_size:]

img_indices_train = (train_idx // 5).to(device)
img_indices_val = (val_idx // 5).to(device)

tx_val_t, im_val_t = tx_train_t[val_idx], im_train_exp[val_idx]
tx_train_t_sub, im_train_exp_sub = tx_train_t[train_idx], im_train_exp[train_idx]

train_dataset = TensorDataset(tx_train_t_sub, im_train_exp_sub, img_indices_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
print(f"Train pairs: {len(train_dataset)}, Validation pairs: {len(val_idx)}")

# ------------------------------------------------------
# 7. Recall@K utility (Unchanged)
# ------------------------------------------------------
@torch.no_grad()
def recall_at_k(model, tx_queries, im_database, query_img_indices, repeat_factor=5, ks=(1, 5, 10, 50), k_csls=10):
    model.eval()
    preds_list = []
    # Process in chunks to avoid OOM on validation
    for i in range(0, len(tx_queries), 1024):
        chunk = tx_queries[i:i+1024].to(device)
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0)

    im_database = im_database.to(device)
    query_img_indices = query_img_indices.to(device)

    # CSLS calculation
    sim = preds @ im_database.T

    knn_q = torch.topk(sim, k=k_csls, dim=1).values
    mean_knn_q = knn_q.mean(1, keepdim=True)

    knn_d = torch.topk(sim.T, k=k_csls, dim=1).values
    mean_knn_d = knn_d.mean(1, keepdim=True).T

    csls_sim = 2 * sim - mean_knn_q - mean_knn_d

    gt = query_img_indices

    top_indices = torch.argsort(csls_sim, dim=1, descending=True)

    recalls = {}
    for k in ks:
        top_k_preds = top_indices[:, :k]
        correct_in_top_k = (top_k_preds == gt.unsqueeze(1)).any(dim=1)
        recall_at_k = correct_in_top_k.float().mean().item()
        recalls[f"R@{k}"] = recall_at_k

    return recalls

# ------------------------------------------------------
# 8. Training Loop (THE "FINAL FINE-TUNE" RUN)
# ------------------------------------------------------
EPOCHS = 50 # Max epochs for fine-tuning
START_LR = 1e-5 # Start with a low learning rate
WEIGHT_DECAY = 5e-5
LOAD_PATH = "marathon_run_best.pth"
SAVE_PATH = "final_finetune_best.pth" # New save path

val_query_subset = tx_val_t[:5000]
val_indices_subset = img_indices_val[:5000]
val_db_subset = im_train_t_unique

# --- Initialize WINNING model ---
model = ResidualTranslator(
    R_init_shape=R_SHAPE,
    input_dim=1024,
    hidden_dim=1024, # Our 0.83359 winner's capacity
    output_dim=1536
).to(device)

# --- Load our best-ever model ---
try:
    model.load_state_dict(torch.load(LOAD_PATH))
    print(f"✅ Loaded weights from '{LOAD_PATH}'. Starting fine-tune.")
except Exception as e:
    print(f"FATAL: Could not load '{LOAD_PATH}'. Stopping. Error: {e}")
    # Stop the script if we can't load the model
    raise e

best_r10 = 0.5672 # Start from our marathon-run's best score

optimizer = torch.optim.AdamW(model.parameters(), lr=START_LR, weight_decay=WEIGHT_DECAY)

# --- This is the new, smart scheduler ---
# It will monitor the R@10 score ('max' mode)
# 'patience=5': Wait 5 epochs for an improvement
# 'factor=0.5': If no improvement, cut LR in half
# 'min_lr=1e-7': Don't go lower than this
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 'max',
    patience=5,
    factor=0.5,
    min_lr=1e-7
    # verbose=True was removed, it is deprecated.
)
scaler = torch.cuda.amp.GradScaler() # Mixed precision

print(f"\nTraining Final Fine-Tune (Epochs: {EPOCHS}, Start LR={START_LR})...\n")

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0

    for x_batch, y_batch, img_indices in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            y_pred = model(x_batch)

            # --- Our proven winning loss combo ---
            sims_with_tau = y_pred @ y_batch.T / TAU
            labels = torch.arange(y_pred.size(0), device=device)
            loss_con = F.cross_entropy(sims_with_tau, labels)

            with torch.no_grad():
                sims_no_tau = y_pred @ y_batch.T
                positive_mask = (img_indices.unsqueeze(1) == img_indices.unsqueeze(0))
                sims_no_tau.masked_fill_(positive_mask, -float('inf'))
                hard_neg_idx = sims_no_tau.argmax(dim=1)

            y_hard_neg = y_batch[hard_neg_idx]
            loss_tri = triplet_loss_fn(y_pred, y_batch, y_hard_neg)

            loss = (LOSS_WEIGHT_CONTRASTIVE * loss_con) + (LOSS_WEIGHT_TRIPLET * loss_tri)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    # ----- Validation (MUST run every epoch for the scheduler) -----
    rec = recall_at_k(model, val_query_subset, val_db_subset, val_indices_subset)
    current_r10 = rec['R@10']

    # --- THIS IS THE CORRECTED PRINT STATEMENT ---
    print(f"Epoch {epoch:02d}: loss={avg_loss:.4f} | R@1(Val)={rec['R@1']:.4f}  R@5(Val)={rec['R@5']:.4f}  R@10(Val)={current_r10:.4f}  R@50(Val)={rec['R@50']:.4f}")

    if current_r10 > best_r10:
        best_r10 = current_r10
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"✅ Best model saved (epoch {epoch}, R@10={best_r10:.4f})")

    # Step the scheduler based on the R@10 score
    scheduler.step(current_r10)

    # Manually print the LR to replace 'verbose'
    current_lr = optimizer.param_groups[0]['lr']
    if 'last_lr' not in locals(): locals()['last_lr'] = START_LR
    if current_lr != locals()['last_lr']:
        print(f"--- Learning rate reduced to {current_lr:.1e} ---")
        locals()['last_lr'] = current_lr

    # Check if LR is at minimum (early stopping)
    if current_lr <= (1e-7 + 1e-9):
        print(f"Learning rate at minimum. Stopping training early at epoch {epoch}.")
        break

# --- This block is now DE-DENTED ---
print(f"\n🎯 Best R@10 on validation = {best_r10:.4f}\nModel saved as {SAVE_PATH}")

# ------------------------------------------------------
# 9. Inference + Submission
# ------------------------------------------------------
print(f"Loading best model from {SAVE_PATH} for inference...")
model = ResidualTranslator(
    R_init_shape=R_SHAPE,
    input_dim=1024,
    hidden_dim=1024,
    output_dim=1536
).to(device)
model.load_state_dict(torch.load(SAVE_PATH))
model.eval()
print("Best model loaded.")

with torch.no_grad():
    preds_list = []
    for i in range(0, len(tx_test_t), 1024):
        chunk = tx_test_t[i:i+1024].to(device)
        preds_list.append(model(chunk))
    preds = torch.cat(preds_list, dim=0).cpu().numpy()

test_ids = test_data["captions/ids"].astype(int)
submission = pd.DataFrame({
    "id": test_ids,
    "embedding": [list(map(float, row)) for row in preds]
})
submission.to_csv("submission_final_finetune.csv", index=False)
print("\n✅ submission_final_finetune.csv saved successfully.")



Using device: cuda
Train shapes: (125000, 1024), (25000, 1536), (125000, 1536)
Test shape: (1500, 1024)


/tmp/ipython-input-2478858331.py:178: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() # Mixed precision


Data preprocessed and normalized.
Base R shape: (1536, 1024)
Train pairs: 112500, Validation pairs: 12500
✅ Loaded weights from 'marathon_run_best.pth'. Starting fine-tune.

Training Final Fine-Tune (Epochs: 50, Start LR=1e-05)...



Epoch 1/50:   0%|          | 0/220 [00:00<?, ?it/s]/tmp/ipython-input-2478858331.py:189: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 01: loss=1.0723 | R@1(Val)=0.2684  R@5(Val)=0.5124  R@10(Val)=0.6172  R@50(Val)=0.8304
✅ Best model saved (epoch 1, R@10=0.6172)


...

Epoch 39: loss=1.0665 | R@1(Val)=0.2656  R@5(Val)=0.5082  R@10(Val)=0.6124  R@50(Val)=0.8294


Epoch 40: loss=1.0682 | R@1(Val)=0.2656  R@5(Val)=0.5082  R@10(Val)=0.6124  R@50(Val)=0.8294


Epoch 41: loss=1.0672 | R@1(Val)=0.2656  R@5(Val)=0.5082  R@10(Val)=0.6122  R@50(Val)=0.8294


Epoch 42: loss=1.0669 | R@1(Val)=0.2656  R@5(Val)=0.5082  R@10(Val)=0.6124  R@50(Val)=0.8294


Epoch 43: loss=1.0669 | R@1(Val)=0.2656  R@5(Val)=0.5082  R@10(Val)=0.6124  R@50(Val)=0.8294
--- Learning rate reduced to 1.0e-07 ---
Learning rate at minimum. Stopping training early at epoch 43.

🎯 Best R@10 on validation = 0.6172
Model saved as final_finetune_best.pth
Loading best model from final_finetune_best.pth for inference...
Best model loaded.

✅ submission_final_finetune.csv saved successfully.
